In [ ]:
from pathlib import Path

# single source of truth for dashboard outputs
OUTPUT_DIR = Path("docs/data/derived")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

def save_csv(df, name):
    p = OUTPUT_DIR / name
    df.to_csv(p, index=False)
    print(f"✔ saved: {p}")

# …then everywhere you save:
# save_csv(provider_scores_df, "provider_scores_latest.csv")
# save_csv(roi_df,              "roi_comparison.csv")


In [1]:
#scrape provider
# --- Async helper that works in notebooks and GitHub Actions ---
import asyncio

def await_safe(coro):
    """
    Run an async coroutine from anywhere:
    - If an event loop is already running (Jupyter/nbconvert), use nest_asyncio + run_until_complete
    - Otherwise, use asyncio.run
    """
    try:
        loop = asyncio.get_event_loop()
        if loop.is_running():
            try:
                import nest_asyncio
                nest_asyncio.apply()
            except Exception:
                pass
            return loop.run_until_complete(coro)
        else:
            return asyncio.run(coro)
    except RuntimeError:
        # No current loop
        return asyncio.run(coro)


In [5]:
# Lambda Labs 

import re, requests, pandas as pd
from typing import Optional
from bs4 import BeautifulSoup
from datetime import datetime, timezone

HEADERS = {"User-Agent": "Mozilla/5.0"}

SLIM_COLS = [
    "provider","region","gpu_model","type","duration","gpu_count",
    "price_hourly_usd","source_url","fetched_at_utc"
]

def _now_iso() -> str:
    return datetime.now(timezone.utc).isoformat(timespec="seconds")

def _ensure_slim(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for c in SLIM_COLS:
        if c not in out.columns: out[c] = None
    out["price_hourly_usd"] = pd.to_numeric(out["price_hourly_usd"], errors="coerce")
    out["fetched_at_utc"] = pd.to_datetime(out["fetched_at_utc"], errors="coerce", utc=True).dt.tz_convert(None)
    return out[SLIM_COLS]

def _norm_gpu(s: str) -> str:
    s = re.sub(r"\bon[-\s]?demand\b", "", s, flags=re.I)
    s = s.replace("NVIDIA", "").strip()
    s = re.sub(r"\s+", " ", s)
    s = s.upper().replace("GH200", "H200")  # treat GH200 as H200
    return s.strip()

def _gpu_count(s: str) -> Optional[int]:
    if not isinstance(s, str): 
        return None
    m = re.search(r"(\d+)x", s, flags=re.I)
    return int(m.group(1)) if m else None

def _price_in(text: str) -> Optional[float]:
    if not isinstance(text, str): 
        return None
    m = re.search(r"\$\s*([0-9]+(?:\.[0-9]+)?)", text.replace(",", ""))
    return float(m.group(1)) if m else None

def _infer_region(table) -> str:
    hdr = table.find_previous(["h2","h3","h4","p"])
    if hdr:
        t = hdr.get_text(" ", strip=True).lower()
        if "europe" in t or "eu" in t: return "EU"
        if "united states" in t or "us" in t or "usa" in t: return "US"
    return "US"

def scrape_lambda_labs(region: Optional[str] = None) -> pd.DataFrame:
    """
    Scrapes https://cloud.lambdalabs.com/pricing and returns SLIM rows
    for H100/H200 (On-Demand, 1h). If `region` provided, overrides detected region.
    """
    url = "https://cloud.lambdalabs.com/pricing"
    r = requests.get(url, headers=HEADERS, timeout=30)
    r.raise_for_status()
    soup = BeautifulSoup(r.text, "html.parser")

    rows_out = []
    tables = soup.find_all("table")
    for table in tables:
        tbl_region = region or _infer_region(table)
        for tr in table.find_all("tr"):
            tds = [td.get_text(" ", strip=True) for td in tr.find_all("td")]
            if not tds:
                continue
            row_text = " | ".join(tds)

            if not (re.search(r"\bH100\b", row_text, re.I) or re.search(r"\bH200\b|\bGH200\b", row_text, re.I)):
                continue

            price = _price_in(row_text)
            if price is None:
                continue

            
            gpu_cell = next((c for c in tds if ("H100" in c.upper() or "H200" in c.upper() or "GH200" in c.upper())), None)
            gpu_model = _norm_gpu(gpu_cell or ("H100" if "H100" in row_text.upper() else "H200"))
            count = _gpu_count(gpu_model)

            rows_out.append({
                "provider": "Lambda Labs",
                "region": tbl_region,
                "gpu_model": gpu_model,      
                "type": "On-Demand",
                "duration": "1h",
                "gpu_count": count,
                "price_hourly_usd": price,
                "source_url": url,
                "fetched_at_utc": _now_iso(),
            })

    df = pd.DataFrame(rows_out)
    if df.empty:
        return _ensure_slim(df)
    keep = df["gpu_model"].str.contains(r"\bH100\b|\bH200\b", regex=True, na=False)
    df = df[keep].reset_index(drop=True)
    return _ensure_slim(df)

# Example:
df_lambda = scrape_lambda_labs(region="US")
display(df_lambda.head())



,provider,region,gpu_model,type,duration,gpu_count,price_hourly_usd,source_url,fetched_at_utc
0,Lambda Labs,US,8X H100 SXM,On-Demand,1h,8,2.99,https://cloud.lambdalabs.com/pricing,2025-09-04 11:15:09
1,Lambda Labs,US,4X H100 SXM,On-Demand,1h,4,3.09,https://cloud.lambdalabs.com/pricing,2025-09-04 11:15:09
2,Lambda Labs,US,2X H100 SXM,On-Demand,1h,2,3.19,https://cloud.lambdalabs.com/pricing,2025-09-04 11:15:09
3,Lambda Labs,US,1X H200,On-Demand,1h,1,1.49,https://cloud.lambdalabs.com/pricing,2025-09-04 11:15:09
4,Lambda Labs,US,1X H100 SXM,On-Demand,1h,1,3.29,https://cloud.lambdalabs.com/pricing,2025-09-04 11:15:09


In [7]:
# ===== Lambda Labs (static) + RunPod (async) with per-provider history =====
# Slim schema: provider, region, gpu_model, type, duration, gpu_count,
#              price_hourly_usd, source_url, fetched_at_utc
# Py 3.8 compatible

import re, os, asyncio, pandas as pd, tempfile
from typing import Optional, Dict, Any
from bs4 import BeautifulSoup
from datetime import datetime, timezone
from pathlib import Path
import requests

HEADERS = {"User-Agent": "Mozilla/5.0"}
SLIM_COLS = [
    "provider","region","gpu_model","type","duration","gpu_count",
    "price_hourly_usd","source_url","fetched_at_utc"
]

# -------- storage (per-provider history/snapshots) --------
BASE = Path("docs/data")
HIST_DIR = BASE / "history"
SNAP_DIR = BASE / "snapshots"
LATEST_DIR = BASE / "latest"
for d in (HIST_DIR, SNAP_DIR, LATEST_DIR):
    try:
        d.mkdir(parents=True, exist_ok=True)
    except Exception:
        # fallback to tmp if workspace is read-only
        tmp = Path(tempfile.gettempdir()) / "gpu_data"
        d = tmp / d.name
        d.mkdir(parents=True, exist_ok=True)

def _now_iso() -> str:
    return datetime.now(timezone.utc).isoformat(timespec="seconds")

def _ensure_slim(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for c in SLIM_COLS:
        if c not in out.columns: out[c] = None
    out["price_hourly_usd"] = pd.to_numeric(out["price_hourly_usd"], errors="coerce")
    out["fetched_at_utc"] = pd.to_datetime(out["fetched_at_utc"], errors="coerce", utc=True).dt.tz_convert(None)
    return out[SLIM_COLS]

def _save_provider(df: pd.DataFrame, provider_slug: str):
    df = _ensure_slim(df)
    ts = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M%S")
    # snapshot
    snap_path = SNAP_DIR / f"{ts}_{provider_slug}.csv"
    try:
        df.to_csv(snap_path, index=False)
    except Exception:
        snap_path = Path(tempfile.gettempdir()) / f"{ts}_{provider_slug}.csv"
        df.to_csv(snap_path, index=False)
    # history (append + dedupe)
    hist_path = HIST_DIR / f"{provider_slug}_history.csv"
    if hist_path.exists():
        old = pd.read_csv(hist_path, low_memory=False)
        old["fetched_at_utc"] = pd.to_datetime(old["fetched_at_utc"], errors="coerce", utc=True).dt.tz_convert(None)
        all_df = pd.concat([old, df], ignore_index=True)
    else:
        all_df = df.copy()
    all_df = (all_df
              .dropna(subset=["gpu_model","price_hourly_usd"])
              .drop_duplicates(subset=["provider","region","gpu_model","type","duration",
                                       "fetched_at_utc","price_hourly_usd"], keep="last")
              .sort_values("fetched_at_utc"))
    try:
        all_df.to_csv(hist_path, index=False)
    except Exception:
        hist_path = Path(tempfile.gettempdir()) / f"{provider_slug}_history.csv"
        all_df.to_csv(hist_path, index=False)
    # latest (newest rows only per gpu/type/region/duration)
    key = ["gpu_model","type","region","duration"]
    latest = all_df.sort_values("fetched_at_utc").drop_duplicates(subset=key, keep="last")
    latest_path = LATEST_DIR / f"{provider_slug}_latest.csv"
    try:
        latest.to_csv(latest_path, index=False)
    except Exception:
        latest_path = Path(tempfile.gettempdir()) / f"{provider_slug}_latest.csv"
        latest.to_csv(latest_path, index=False)
    print(f"[{provider_slug}] snapshot -> {snap_path}\n[{provider_slug}] history  -> {hist_path}\n[{provider_slug}] latest   -> {latest_path}")
    return latest

# ------------------------- Lambda Labs (static) -------------------------
def _norm_gpu_lambda(s: str) -> str:
    s = re.sub(r"\bon[-\s]?demand\b", "", s, flags=re.I)
    s = s.replace("NVIDIA", "").strip()
    s = re.sub(r"\s+", " ", s)
    s = s.upper().replace("GH200", "H200")
    return s.strip()

def _gpu_count(text: str) -> Optional[int]:
    if not isinstance(text, str): return None
    m = re.search(r"(\d+)\s*x", text, flags=re.I)
    return int(m.group(1)) if m else None

def _price_dollar(text: str) -> Optional[float]:
    if not isinstance(text, str): return None
    m = re.search(r"\$\s*([0-9]+(?:\.[0-9]+)?)", text.replace(",", ""))
    return float(m.group(1)) if m else None

def _infer_region_lambda(table) -> str:
    hdr = table.find_previous(["h2","h3","h4","p"])
    if hdr:
        t = hdr.get_text(" ", strip=True).lower()
        if "europe" in t or "eu" in t: return "EU"
        if "united states" in t or "us" in t or "usa" in t: return "US"
    return "US"

def scrape_lambda_labs(region: Optional[str] = None) -> pd.DataFrame:
    url = "https://cloud.lambdalabs.com/pricing"
    r = requests.get(url, headers=HEADERS, timeout=30); r.raise_for_status()
    soup = BeautifulSoup(r.text, "html.parser")

    out = []
    for table in soup.find_all("table"):
        tbl_region = region or _infer_region_lambda(table)
        for tr in table.find_all("tr"):
            tds = [td.get_text(" ", strip=True) for td in tr.find_all("td")]
            if not tds: continue
            row_text = " | ".join(tds)
            if not (re.search(r"\bH100\b", row_text, re.I) or re.search(r"\bH200\b|\bGH200\b", row_text, re.I)):
                continue
            price = _price_dollar(row_text)
            if price is None: continue
            gpu_cell = next((c for c in tds if ("H100" in c.upper() or "H200" in c.upper() or "GH200" in c.upper())), None)
            model = _norm_gpu_lambda(gpu_cell or ("H100" if "H100" in row_text.upper() else "H200"))
            out.append({
                "provider": "Lambda Labs",
                "region": tbl_region,
                "gpu_model": model,
                "type": "On-Demand",
                "duration": "1h",
                "gpu_count": _gpu_count(model),
                "price_hourly_usd": price,
                "source_url": url,
                "fetched_at_utc": _now_iso(),
            })
    df = pd.DataFrame(out)
    if df.empty: return _ensure_slim(df)
    keep = df["gpu_model"].str.contains(r"\bH100\b|\bH200\b", regex=True, na=False)
    return _ensure_slim(df[keep].reset_index(drop=True))

# --------------------------- RunPod (async) ---------------------------
def _extract_gpu_model_runpod(text: str) -> Optional[str]:
    if not isinstance(text, str): return None
    text_up = re.sub(r"\s+", " ", text.upper())
    m = re.search(r"(H(?:100|200)(?:\s*(?:SXM|PCIE|NVL))?(?:\s*\d{2,3}\s*GB)?)", text_up)
    return m.group(1).strip() if m else None

def _price_hourly_runpod(text: str) -> Optional[float]:
    if not isinstance(text, str): return None
    t = text.replace(",", "")
    m = re.search(r"\$\s*([0-9]+(?:\.[0-9]+)?)\s*(?:/|\s*(?:per|an)\s*)?(?:h|hr|hour)\b", t, flags=re.I)
    return float(m.group(1)) if m else None

async def scrape_runpod_async() -> pd.DataFrame:
    from playwright.async_api import async_playwright
    url = "https://www.runpod.io/pricing"; region = "Global"

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url, wait_until="domcontentloaded", timeout=60000)
        await page.evaluate("window.scrollTo(0, document.body.scrollHeight)")
        await page.wait_for_timeout(1200)
        html = await page.content()
        await browser.close()

    soup = BeautifulSoup(html, "html.parser")
    nodes = soup.find_all(["section","div","article","li","tr"], class_=re.compile(r"(price|pricing|card|grid|table)", re.I))
    if not nodes:
        nodes = soup.find_all(["section","div","article","li","tr","p","span"])

    out = []
    for n in nodes:
        text = n.get_text(" ", strip=True)
        if "H100" not in text and "H200" not in text: 
            continue
        price = _price_hourly_runpod(text)
        if price is None:
            continue
        model = _extract_gpu_model_runpod(text)
        if model is None:
            continue
        out.append({
            "provider": "RunPod",
            "region": region,
            "gpu_model": model,
            "type": "On-Demand",
            "duration": "1h",
            "gpu_count": _gpu_count(text),
            "price_hourly_usd": price,
            "source_url": url,
            "fetched_at_utc": _now_iso(),
        })
    df = pd.DataFrame(out)
    if df.empty: return _ensure_slim(df)
    df = df[df["gpu_model"].str.contains(r"\bH100\b|\bH200\b", na=False)]
    df = df[(df["price_hourly_usd"] > 0) & (df["price_hourly_usd"] < 200)].reset_index(drop=True)
    return _ensure_slim(df)

# --------------- Runner that works in scripts & notebooks ---------------
def arun(coro):
    try:
        loop = asyncio.get_running_loop()
    except RuntimeError:
        return asyncio.run(coro)
    else:
        import nest_asyncio; nest_asyncio.apply()
        return loop.run_until_complete(coro)

# ------------------------------ RUN --------------------------------
# Lambda Labs
df_lambda = scrape_lambda_labs(region="US")
_save_provider(df_lambda, "lambda_labs")

# RunPod
df_runpod = arun(scrape_runpod_async())
_save_provider(df_runpod, "runpod")


[lambda_labs] snapshot -> /var/folders/2_/9wdv7zh56p95l_j0dkkc12zw0000gn/T/20250904_112436_lambda_labs.csv
[lambda_labs] history  -> /var/folders/2_/9wdv7zh56p95l_j0dkkc12zw0000gn/T/lambda_labs_history.csv
[lambda_labs] latest   -> /var/folders/2_/9wdv7zh56p95l_j0dkkc12zw0000gn/T/lambda_labs_latest.csv
[runpod] snapshot -> /var/folders/2_/9wdv7zh56p95l_j0dkkc12zw0000gn/T/20250904_112441_runpod.csv
[runpod] history  -> /var/folders/2_/9wdv7zh56p95l_j0dkkc12zw0000gn/T/runpod_history.csv
[runpod] latest   -> /var/folders/2_/9wdv7zh56p95l_j0dkkc12zw0000gn/T/runpod_latest.csv


,provider,region,gpu_model,type,duration,gpu_count,price_hourly_usd,source_url,fetched_at_utc
4,RunPod,Global,H200 141 GB,On-Demand,1h,None,3.59,https://www.runpod.io/pricing,2025-09-04 11:24:41
6,RunPod,Global,H100 PCIE 80 GB,On-Demand,1h,None,1.99,https://www.runpod.io/pricing,2025-09-04 11:24:41


In [5]:
# --- Nebius H100/H200 scraper (uses YOUR parsing + per-provider history) ---

import re, time, tempfile, requests, pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
from pathlib import Path
import pytz

# ---------- your original config ----------
url = "https://nebius.com/prices"
UA = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124 Safari/537.36"}
TZ = pytz.utc
MIN_PRICE, MAX_PRICE = 0.3, 20.0   # sanity for $/GPU/hr

# ---------- storage dirs (snapshot/history/latest) ----------
SLIM_COLS = [
    "provider","region","gpu_model","type","duration","gpu_count",
    "price_hourly_usd","source_url","fetched_at_utc"
]
BASE = Path("docs/data")
HIST_DIR = BASE / "history"
SNAP_DIR = BASE / "snapshots"
LATEST_DIR = BASE / "latest"
for d in (HIST_DIR, SNAP_DIR, LATEST_DIR):
    try:
        d.mkdir(parents=True, exist_ok=True)
    except Exception:
        pass  # we'll fall back to tmp if write fails later

def _ensure_slim(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for c in SLIM_COLS:
        if c not in out.columns: out[c] = None
    out["price_hourly_usd"] = pd.to_numeric(out["price_hourly_usd"], errors="coerce")
    out["fetched_at_utc"] = pd.to_datetime(out["fetched_at_utc"], errors="coerce", utc=True).dt.tz_convert(None)
    return out[SLIM_COLS]

def _safe_to_csv(df: pd.DataFrame, path: Path):
    try:
        path.parent.mkdir(parents=True, exist_ok=True)
        df.to_csv(path, index=False)
        return path
    except Exception:
        tmp = Path(tempfile.gettempdir()) / path.name
        df.to_csv(tmp, index=False)
        return tmp

def _save_provider(df: pd.DataFrame, provider_slug: str):
    df = _ensure_slim(df)
    ts = datetime.now(TZ).strftime("%Y%m%d_%H%M%S")

    # snapshot
    snap_path = SNAP_DIR / f"{ts}_{provider_slug}.csv"
    snap_path = _safe_to_csv(df, snap_path)

    # history (append + dedupe)
    hist_path = HIST_DIR / f"{provider_slug}_history.csv"
    if hist_path.exists():
        old = pd.read_csv(hist_path, low_memory=False)
        old["fetched_at_utc"] = pd.to_datetime(old["fetched_at_utc"], errors="coerce", utc=True).dt.tz_convert(None)
        all_df = pd.concat([old, df], ignore_index=True)
    else:
        all_df = df.copy()
    all_df = (all_df
              .dropna(subset=["gpu_model","price_hourly_usd"])
              .drop_duplicates(subset=["provider","region","gpu_model","type","duration",
                                       "fetched_at_utc","price_hourly_usd"], keep="last")
              .sort_values("fetched_at_utc"))
    hist_path = _safe_to_csv(all_df, hist_path)

    # latest (newest per gpu/type/region/duration)
    key = ["gpu_model","type","region","duration"]
    latest = all_df.sort_values("fetched_at_utc").drop_duplicates(subset=key, keep="last")
    latest_path = LATEST_DIR / f"{provider_slug}_latest.csv"
    latest_path = _safe_to_csv(latest, latest_path)

    print(f"[{provider_slug}] snapshot -> {snap_path}\n[{provider_slug}] history  -> {hist_path}\n[{provider_slug}] latest   -> {latest_path}")
    return latest

# ---------- your original parsing (unchanged) ----------
def find_price_strict(text: str):
    """Match $X/hr, $X per hour, $X/hour, case-insensitive."""
    if not text: return None
    m = re.search(r"\$([0-9]+(?:\.[0-9]+)?)\s*(?:/|\s*per\s*)?\s*(?:h|hr|hour)\b", text, flags=re.I)
    return float(m.group(1)) if m else None

def parse_nebius_from_html(html: str) -> dict:
    """Return {'H100': price, 'H200': price} if found."""
    soup = BeautifulSoup(html, "html.parser")
    results = {}

    # Focus on plausible pricing containers first (tables / pricing sections)
    blocks = []
    blocks.extend(soup.find_all("table"))
    if not blocks:
        blocks.extend(soup.find_all(["section","div"], class_=re.compile("price|pricing|compute", re.I)))
    if not blocks:
        blocks = soup.find_all(["div","tr","li","p","span"])

    for blk in blocks:
        t = blk.get_text(" ", strip=True)
        if not t: continue

        has_h100 = bool(re.search(r"\bH100\b", t, flags=re.I))
        has_h200 = bool(re.search(r"\bH200\b", t, flags=re.I))
        if not (has_h100 or has_h200): continue

        price = find_price_strict(t)
        if price is None or not (MIN_PRICE <= price <= MAX_PRICE): continue

        if has_h100 and "H100" not in results:
            results["H100"] = price
        if has_h200 and "H200" not in results:
            results["H200"] = price

        if len(results) == 2:
            break

    return results

# ---------- run (your flow) ----------
html = None
try:
    r = requests.get(url, headers=UA, timeout=30)
    if r.status_code == 200 and r.text:
        html = r.text
except Exception:
    html = None

results = {}
if html:
    results = parse_nebius_from_html(html)

# Optional Playwright fallback if nothing found
if not results:
    try:
        from playwright.sync_api import sync_playwright
        with sync_playwright() as p:
            browser = p.chromium.launch(headless=True)
            page = browser.new_page()
            page.goto(url, wait_until="networkidle", timeout=60000)
            page.wait_for_timeout(2000)  # allow dynamic content
            html_pw = page.content()
            browser.close()
        results = parse_nebius_from_html(html_pw)
    except Exception:
        pass  # proceed with whatever we have

# ---------- map YOUR results -> SLIM schema & save ----------
rows = []
ts = datetime.now(TZ).isoformat()
for gpu, price in results.items():
    rows.append({
        "provider": "Nebius",
        "region": "Global",            # keep simple; refine if you later detect regions
        "gpu_model": gpu,              # map gpu_type -> gpu_model
        "type": "On-Demand",
        "duration": "1h",
        "gpu_count": None,
        "price_hourly_usd": price,     # map on_demand_price -> price_hourly_usd
        "source_url": url,
        "fetched_at_utc": ts,          # map scraped_at -> fetched_at_utc
    })

df_nebius = pd.DataFrame(rows, columns=SLIM_COLS)
latest = _save_provider(df_nebius, "nebius")
print(f"Nebius rows this run: {len(df_nebius)}")
display(df_nebius.head())


[nebius] snapshot -> docs/data/snapshots/20250904_114258_nebius.csv
[nebius] history  -> docs/data/history/nebius_history.csv
[nebius] latest   -> docs/data/latest/nebius_latest.csv
Nebius rows this run: 2


,provider,region,gpu_model,type,duration,gpu_count,price_hourly_usd,source_url,fetched_at_utc
0,Nebius,Global,H200,On-Demand,1h,None,2.3,https://nebius.com/prices,2025-09-04T11:42:58.443649+00:00
1,Nebius,Global,H100,On-Demand,1h,None,2.0,https://nebius.com/prices,2025-09-04T11:42:58.443649+00:00


In [6]:
# ================= VoltagePark (async) — slim schema + per-provider history =================
# Slim schema: provider, region, gpu_model, type, duration, gpu_count,
#              price_hourly_usd, source_url, fetched_at_utc
# Py 3.8 compatible

import re, asyncio, tempfile, pandas as pd
from datetime import datetime, timezone
from pathlib import Path
from typing import Optional
from playwright.async_api import async_playwright

# ---- storage + schema helpers (same as other providers) ----
SLIM_COLS = [
    "provider","region","gpu_model","type","duration","gpu_count",
    "price_hourly_usd","source_url","fetched_at_utc"
]
BASE = Path("docs/data")
HIST_DIR = BASE / "history"
SNAP_DIR = BASE / "snapshots"
LATEST_DIR = BASE / "latest"
for d in (HIST_DIR, SNAP_DIR, LATEST_DIR):
    try:
        d.mkdir(parents=True, exist_ok=True)
    except Exception:
        pass  # fallback handled in _safe_to_csv

def _now_iso() -> str:
    return datetime.now(timezone.utc).isoformat(timespec="seconds")

def _ensure_slim(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for c in SLIM_COLS:
        if c not in out.columns:
            out[c] = None
    out["price_hourly_usd"] = pd.to_numeric(out["price_hourly_usd"], errors="coerce")
    out["fetched_at_utc"] = pd.to_datetime(out["fetched_at_utc"], errors="coerce", utc=True).dt.tz_convert(None)
    return out[SLIM_COLS]

def _safe_to_csv(df: pd.DataFrame, path: Path):
    try:
        path.parent.mkdir(parents=True, exist_ok=True)
        df.to_csv(path, index=False)
        return path
    except Exception:
        tmp = Path(tempfile.gettempdir()) / path.name
        df.to_csv(tmp, index=False)
        return tmp

def _save_provider(df: pd.DataFrame, provider_slug: str):
    df = _ensure_slim(df)
    ts = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M%S")

    # snapshot
    snap_path = SNAP_DIR / f"{ts}_{provider_slug}.csv"
    snap_path = _safe_to_csv(df, snap_path)

    # history (append + dedupe)
    hist_path = HIST_DIR / f"{provider_slug}_history.csv"
    if hist_path.exists():
        old = pd.read_csv(hist_path, low_memory=False)
        old["fetched_at_utc"] = pd.to_datetime(old["fetched_at_utc"], errors="coerce", utc=True).dt.tz_convert(None)
        all_df = pd.concat([old, df], ignore_index=True)
    else:
        all_df = df.copy()
    all_df = (
        all_df.dropna(subset=["gpu_model","price_hourly_usd"])
              .drop_duplicates(subset=["provider","region","gpu_model","type","duration",
                                       "fetched_at_utc","price_hourly_usd"], keep="last")
              .sort_values("fetched_at_utc")
    )
    hist_path = _safe_to_csv(all_df, hist_path)

    # latest (newest per gpu/type/region/duration)
    key = ["gpu_model","type","region","duration"]
    latest = all_df.sort_values("fetched_at_utc").drop_duplicates(subset=key, keep="last")
    latest_path = LATEST_DIR / f"{provider_slug}_latest.csv"
    latest_path = _safe_to_csv(latest, latest_path)

    print(f"[{provider_slug}] snapshot -> {snap_path}\n[{provider_slug}] history  -> {hist_path}\n[{provider_slug}] latest   -> {latest_path}")
    return latest

# ---- YOUR scraping logic, adapted to slim schema ----
async def scrape_voltagepark() -> pd.DataFrame:
    url = "https://dashboard.voltagepark.com/order/configure-deployment"
    rows = []
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url, timeout=60000)
        await page.wait_for_timeout(5000)

        html = await page.content()
        await browser.close()

    for line in html.splitlines():
        if ("H100" in line or "H200" in line) and "$" in line:
            try:
                # your original pattern
                m = re.search(r"\$?(\d+(?:\.\d+)?)(?=/GPU/hour)", line)
                if m:
                    price = float(m.group(1))
                    gpu = "H100" if "H100" in line else "H200"
                    rows.append({
                        "provider": "VoltagePark",
                        "region": "US",
                        "gpu_model": gpu,
                        "type": "On-Demand",
                        "duration": "1h",
                        "gpu_count": None,
                        "price_hourly_usd": price,
                        "source_url": url,
                        "fetched_at_utc": _now_iso(),
                    })
            except Exception as e:
                # keep silent in prod; print minimal context if you want
                # print(f"[VoltagePark Parse Error] {e}")
                pass

    df = pd.DataFrame(rows, columns=SLIM_COLS)
    if df.empty:
        return _ensure_slim(df)

    # dedupe by (gpu_model, price)
    df = (df.sort_values(["gpu_model","price_hourly_usd","fetched_at_utc"])
            .drop_duplicates(subset=["gpu_model","price_hourly_usd"], keep="last")
            .reset_index(drop=True))

    # sanity: plausible $/hr range
    df = df[(df["price_hourly_usd"] > 0) & (df["price_hourly_usd"] < 200)]
    return _ensure_slim(df)

# ---- runner that works in notebooks & scripts ----
def arun(coro):
    try:
        loop = asyncio.get_running_loop()
    except RuntimeError:
        return asyncio.run(coro)
    else:
        import nest_asyncio; nest_asyncio.apply()
        return loop.run_until_complete(coro)

# ------------------------------ RUN --------------------------------
df_voltage = arun(scrape_voltagepark())
_save_provider(df_voltage, "voltagepark")
display(df_voltage.head())


[voltagepark] snapshot -> docs/data/snapshots/20250904_123626_voltagepark.csv
[voltagepark] history  -> docs/data/history/voltagepark_history.csv
[voltagepark] latest   -> docs/data/latest/voltagepark_latest.csv


,provider,region,gpu_model,type,duration,gpu_count,price_hourly_usd,source_url,fetched_at_utc
0,VoltagePark,US,H100,On-Demand,1h,None,1.99,https://dashboard.voltagepark.com/order/config...,2025-09-04 12:36:26


In [8]:
# ================= Vast.ai (async) — slim schema + per-provider history =================
# Slim schema: provider, region, gpu_model, type, duration, gpu_count,
#              price_hourly_usd, source_url, fetched_at_utc
# Py 3.8 compatible

import re, asyncio, pandas as pd, tempfile
from datetime import datetime, timezone
from pathlib import Path
from playwright.async_api import async_playwright

# -------- storage + schema helpers (same as other providers) --------
SLIM_COLS = [
    "provider","region","gpu_model","type","duration","gpu_count",
    "price_hourly_usd","source_url","fetched_at_utc"
]
BASE = Path("docs/data")
HIST_DIR = BASE / "history"
SNAP_DIR = BASE / "snapshots"
LATEST_DIR = BASE / "latest"
for d in (HIST_DIR, SNAP_DIR, LATEST_DIR):
    try:
        d.mkdir(parents=True, exist_ok=True)
    except Exception:
        pass  # fallback handled below

def _now_iso() -> str:
    return datetime.now(timezone.utc).isoformat(timespec="seconds")

def _ensure_slim(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for c in SLIM_COLS:
        if c not in out.columns:
            out[c] = None
    out["price_hourly_usd"] = pd.to_numeric(out["price_hourly_usd"], errors="coerce")
    out["fetched_at_utc"] = pd.to_datetime(out["fetched_at_utc"], errors="coerce", utc=True).dt.tz_convert(None)
    return out[SLIM_COLS]

def _safe_to_csv(df: pd.DataFrame, path: Path):
    try:
        path.parent.mkdir(parents=True, exist_ok=True)
        df.to_csv(path, index=False)
        return path
    except Exception:
        tmp = Path(tempfile.gettempdir()) / path.name
        df.to_csv(tmp, index=False)
        return tmp

def _save_provider(df: pd.DataFrame, provider_slug: str):
    df = _ensure_slim(df)
    ts = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M%S")

    # snapshot
    snap_path = SNAP_DIR / f"{ts}_{provider_slug}.csv"
    snap_path = _safe_to_csv(df, snap_path)

    # history (append + dedupe)
    hist_path = HIST_DIR / f"{provider_slug}_history.csv"
    if hist_path.exists():
        old = pd.read_csv(hist_path, low_memory=False)
        old["fetched_at_utc"] = pd.to_datetime(old["fetched_at_utc"], errors="coerce", utc=True).dt.tz_convert(None)
        all_df = pd.concat([old, df], ignore_index=True)
    else:
        all_df = df.copy()
    all_df = (
        all_df.dropna(subset=["gpu_model","price_hourly_usd"])
              .drop_duplicates(subset=["provider","region","gpu_model","type","duration",
                                       "fetched_at_utc","price_hourly_usd"], keep="last")
              .sort_values("fetched_at_utc")
    )
    hist_path = _safe_to_csv(all_df, hist_path)

    # latest (newest per gpu/type/region/duration)
    key = ["gpu_model","type","region","duration"]
    latest = all_df.sort_values("fetched_at_utc").drop_duplicates(subset=key, keep="last")
    latest_path = LATEST_DIR / f"{provider_slug}_latest.csv"
    latest_path = _safe_to_csv(latest, latest_path)

    print(f"[{provider_slug}] snapshot -> {snap_path}\n[{provider_slug}] history  -> {hist_path}\n[{provider_slug}] latest   -> {latest_path}")
    return latest

# -------- YOUR scraping logic, adapted to slim schema --------
async def scrape_vast_products() -> pd.DataFrame:
    url = "https://vast.ai/products/gpu-cloud"
    rows = []
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url, timeout=60000)
        # Try to reveal lazy content
        await page.evaluate("window.scrollTo(0, document.body.scrollHeight)")
        await page.wait_for_timeout(1500)
        await page.evaluate("window.scrollTo(0, 0)")
        await page.wait_for_timeout(500)

        content = await page.content()
        await browser.close()

    # Your original approach: scan lines and pick $ numbers near H100/H200
    for line in content.splitlines():
        if ("H100" in line or "H200" in line) and "$" in line:
            try:
                gpu_model = "H100" if "H100" in line else "H200"
                # pull all $-bearing tokens in the line
                dollars = [s for s in re.split(r"\s+", line) if "$" in s]
                price_val = None
                for token in dollars:
                    clean = "".join(c for c in token if c.isdigit() or c == ".")
                    if not clean:
                        continue
                    price = float(clean)
                    if 0.1 < price < 100:  # sanity filter like you had
                        price_val = price
                        break
                if price_val is None:
                    continue

                rows.append({
                    "provider": "Vast.ai",
                    "region": "Global",
                    "gpu_model": gpu_model,
                    "type": "On-Demand",
                    "duration": "1h",
                    "gpu_count": None,
                    "price_hourly_usd": price_val,
                    "source_url": url,
                    "fetched_at_utc": _now_iso(),
                })
            except Exception:
                # swallow parse errors to keep the run clean
                pass

    df = pd.DataFrame(rows, columns=SLIM_COLS)
    if df.empty:
        return _ensure_slim(df)

    # Deduplicate by (gpu_model, price)
    df = (df.sort_values(["gpu_model","price_hourly_usd","fetched_at_utc"])
            .drop_duplicates(subset=["gpu_model","price_hourly_usd"], keep="last")
            .reset_index(drop=True))
    # Sanity clamp
    df = df[(df["price_hourly_usd"] > 0) & (df["price_hourly_usd"] < 200)]
    return _ensure_slim(df)

def arun(coro):
    try:
        loop = asyncio.get_running_loop()
    except RuntimeError:
        return asyncio.run(coro)
    else:
        import nest_asyncio; nest_asyncio.apply()
        return loop.run_until_complete(coro)

df_vastp = arun(scrape_vast_products())
_save_provider(df_vastp, "vastai")
display(df_vastp.head())


[vastai] snapshot -> docs/data/snapshots/20250904_124214_vastai.csv
[vastai] history  -> docs/data/history/vastai_history.csv
[vastai] latest   -> docs/data/latest/vastai_latest.csv


,provider,region,gpu_model,type,duration,gpu_count,price_hourly_usd,source_url,fetched_at_utc
0,Vast.ai,Global,H100,On-Demand,1h,None,1.25,https://vast.ai/products/gpu-cloud,2025-09-04 12:42:14


In [10]:
# ==== Shadeform: precise matcher (nearest-price + hourly hint) ====

import re, asyncio, pandas as pd, tempfile
from datetime import datetime, timezone
from pathlib import Path
from playwright.async_api import async_playwright

# slim schema storage helpers (use the same ones you already have)
SLIM_COLS = [
    "provider","region","gpu_model","type","duration","gpu_count",
    "price_hourly_usd","source_url","fetched_at_utc"
]

def _now_iso():
    return datetime.now(timezone.utc).isoformat(timespec="seconds")

def _ensure_slim(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for c in SLIM_COLS:
        if c not in out.columns: out[c] = None
    out["price_hourly_usd"] = pd.to_numeric(out["price_hourly_usd"], errors="coerce")
    out["fetched_at_utc"] = pd.to_datetime(out["fetched_at_utc"], errors="coerce", utc=True).dt.tz_convert(None)
    return out[SLIM_COLS]

# --------- robust extractors ----------
# require an hourly hint, allowing variants like "/GPU/hour"
PRICE_RE = re.compile(
    r"\$\s*([0-9]+(?:\.[0-9]+)?)\s*(?:/GPU)?\s*(?:/|\s*(?:per|an)\s*)?(?:h|hr|hour)\b",
    re.I
)

GPU_TOKENS = {
    "H100": re.compile(r"\bH100\b", re.I),
    "H200": re.compile(r"\bH200\b", re.I),
    # include B200 so we don't steal its prices
    "_OTHER": re.compile(r"\b(?:B200|H800|A100|A800)\b", re.I),
}

def _find_token_positions(text: str):
    positions = {k: [] for k in GPU_TOKENS.keys()}
    for name, pat in GPU_TOKENS.items():
        for m in pat.finditer(text):
            positions[name].append(m.start())
    return positions

def _find_price_positions(text: str):
    return [(float(m.group(1)), m.start()) for m in PRICE_RE.finditer(text)]

def _nearest_price_to_token(text: str, token: str, window: int = 220):
    """Yield (model, price) pairs by attaching each token occurrence
       to the nearest price with an hourly hint, only if it is closer
       to this token than to any other GPU token."""
    tok_positions = _find_token_positions(text)
    prices = _find_price_positions(text)
    if not tok_positions.get(token) or not prices:
        return []

    # all GPU-ish positions (to compete for 'closeness')
    competitor_positions = []
    for k, pos_list in tok_positions.items():
        if k == token:  # we compare against others later
            continue
        competitor_positions.extend(pos_list)

    rows = []
    for gpos in tok_positions[token]:
        # candidates within a window around the GPU string
        cands = [(price, ppos, abs(ppos - gpos)) for (price, ppos) in prices if abs(ppos - gpos) <= window]
        if not cands:
            continue
        # pick nearest price to this token
        price, ppos, dist = min(cands, key=lambda t: t[2])

        # ensure this price isn't actually closer to another GPU token (e.g., B200)
        if competitor_positions:
            nearest_other = min(abs(ppos - op) for op in competitor_positions)
            if nearest_other < dist:
                continue  # skip: price belongs to another GPU mention

        rows.append((token, price))
    return rows

# --------- scraper ----------
async def scrape_shadeform_rich() -> pd.DataFrame:
    url = "https://www.shadeform.ai/"
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url, timeout=60000, wait_until="domcontentloaded")
        await page.evaluate("window.scrollTo(0, document.body.scrollHeight)")
        await page.wait_for_timeout(900)
        body = await page.inner_text("body")
        await browser.close()

    body = re.sub(r"\s+", " ", body)

    rows = []
    for gpu in ("H100", "H200"):
        for model, price in _nearest_price_to_token(body, gpu, window=220):
            # sanity clamp to avoid accidental captures (tune if needed)
            if not (0.25 <= price <= 8.0):
                continue
            rows.append({
                "provider": "Shadeform",
                "region": "Global",
                "gpu_model": model,
                "type": "On-Demand",
                "duration": "1h",
                "gpu_count": None,
                "price_hourly_usd": price,
                "source_url": url,
                "fetched_at_utc": _now_iso(),
            })

    df = pd.DataFrame(rows, columns=SLIM_COLS)
    if df.empty:
        return _ensure_slim(df)

    # de-dupe (gpu_model, price)
    df = (df.sort_values(["gpu_model","price_hourly_usd","fetched_at_utc"])
            .drop_duplicates(subset=["gpu_model","price_hourly_usd"], keep="last")
            .reset_index(drop=True))
    return _ensure_slim(df)

# ---- runner ----
def arun(coro):
    try:
        loop = asyncio.get_running_loop()
    except RuntimeError:
        return asyncio.run(coro)
    else:
        import nest_asyncio; nest_asyncio.apply()
        return loop.run_until_complete(coro)

# Example:
df_shade = arun(scrape_shadeform_rich())
display(df_shade)


,provider,region,gpu_model,type,duration,gpu_count,price_hourly_usd,source_url,fetched_at_utc
0,Shadeform,Global,H100,On-Demand,1h,None,1.99,https://www.shadeform.ai/,2025-09-04 12:49:12
1,Shadeform,Global,H100,On-Demand,1h,None,2.35,https://www.shadeform.ai/,2025-09-04 12:49:12


In [12]:
# ================= CoreWeave (async) — slim schema + per-provider history =================
# Slim schema: provider, region, gpu_model, type, duration, gpu_count,
#              price_hourly_usd, source_url, fetched_at_utc
# Py 3.8 compatible

import re, asyncio, pandas as pd, tempfile
from datetime import datetime, timezone
from pathlib import Path
from playwright.async_api import async_playwright

# -------- storage + schema helpers --------
SLIM_COLS = [
    "provider","region","gpu_model","type","duration","gpu_count",
    "price_hourly_usd","source_url","fetched_at_utc"
]
BASE = Path("docs/data")
HIST_DIR = BASE / "history"
SNAP_DIR = BASE / "snapshots"
LATEST_DIR = BASE / "latest"
for d in (HIST_DIR, SNAP_DIR, LATEST_DIR):
    try:
        d.mkdir(parents=True, exist_ok=True)
    except Exception:
        pass  # fallback handled in _safe_to_csv

def _now_iso() -> str:
    return datetime.now(timezone.utc).isoformat(timespec="seconds")

def _ensure_slim(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for c in SLIM_COLS:
        if c not in out.columns:
            out[c] = None
    out["price_hourly_usd"] = pd.to_numeric(out["price_hourly_usd"], errors="coerce")
    out["fetched_at_utc"] = pd.to_datetime(out["fetched_at_utc"], errors="coerce", utc=True).dt.tz_convert(None)
    return out[SLIM_COLS]

def _safe_to_csv(df: pd.DataFrame, path: Path):
    try:
        path.parent.mkdir(parents=True, exist_ok=True)
        df.to_csv(path, index=False)
        return path
    except Exception:
        tmp = Path(tempfile.gettempdir()) / path.name
        df.to_csv(tmp, index=False)
        return tmp

def _save_provider(df: pd.DataFrame, provider_slug: str):
    df = _ensure_slim(df)
    ts = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M%S")

    # snapshot
    snap_path = SNAP_DIR / f"{ts}_{provider_slug}.csv"
    snap_path = _safe_to_csv(df, snap_path)

    # history (append + dedupe)
    hist_path = HIST_DIR / f"{provider_slug}_history.csv"
    if hist_path.exists():
        old = pd.read_csv(hist_path, low_memory=False)
        old["fetched_at_utc"] = pd.to_datetime(old["fetched_at_utc"], errors="coerce", utc=True).dt.tz_convert(None)
        all_df = pd.concat([old, df], ignore_index=True)
    else:
        all_df = df.copy()
    all_df = (
        all_df.dropna(subset=["gpu_model","price_hourly_usd"])
              .drop_duplicates(subset=["provider","region","gpu_model","type","duration",
                                       "fetched_at_utc","price_hourly_usd"], keep="last")
              .sort_values("fetched_at_utc")
    )
    hist_path = _safe_to_csv(all_df, hist_path)

    # latest (newest per gpu/type/region/duration)
    key = ["gpu_model","type","region","duration"]
    latest = all_df.sort_values("fetched_at_utc").drop_duplicates(subset=key, keep="last")
    latest_path = LATEST_DIR / f"{provider_slug}_latest.csv"
    latest_path = _safe_to_csv(latest, latest_path)

    print(f"[{provider_slug}] snapshot -> {snap_path}\n[{provider_slug}] history  -> {hist_path}\n[{provider_slug}] latest   -> {latest_path}")
    return latest

# -------- robust token/price matching --------
PRICE_HOURLY_RE = re.compile(
    r"\$\s*([0-9]+(?:\.[0-9]+)?)\s*(?:/|\s*(?:per|an)\s*)?(?:GPU\s*/\s*)?(?:h|hr|hour)\b",
    re.I
)
# Fallback if the site omits 'hr' text; avoid monthly and memory suffixes
PRICE_DOLLAR_RE = re.compile(
    r"\$\s*([0-9]+(?:\.[0-9]+)?)\b(?!\s*(?:k|m|b|/mo|per\s*month|/month|,?\s*GB))",
    re.I
)

GPU_PATS = {
    "H100": re.compile(r"\bH100\b", re.I),
    "H200": re.compile(r"\bH200\b", re.I),
    "_OTHER": re.compile(r"\b(?:B200|A100|A800|H800)\b", re.I),
}

def _find_positions(text: str, pat: re.Pattern):
    return [m.start() for m in pat.finditer(text)]

def _find_prices(text: str, prefer_hourly: bool = True):
    pats = [PRICE_HOURLY_RE] + ([] if not prefer_hourly else [])  # first pass
    prices = [(float(m.group(1)), m.start()) for m in PRICE_HOURLY_RE.finditer(text)]
    if not prices:
        prices = [(float(m.group(1)), m.start()) for m in PRICE_DOLLAR_RE.finditer(text)]
    return prices

def _nearest_prices(text: str, token: str, window: int = 240):
    # Positions of our token vs. competitors
    tok_pos = _find_positions(text, GPU_PATS[token])
    if not tok_pos:
        return []
    comp_pos = []
    for k, pat in GPU_PATS.items():
        if k == token: continue
        comp_pos.extend(_find_positions(text, pat))
    prices = _find_prices(text)
    out = []
    for gpos in tok_pos:
        cands = [(price, ppos, abs(ppos - gpos)) for (price, ppos) in prices if abs(ppos - gpos) <= window]
        if not cands: 
            continue
        price, ppos, dist = min(cands, key=lambda t: t[2])
        if comp_pos:
            nearest_other = min(abs(ppos - op) for op in comp_pos)
            if nearest_other < dist:
                continue
        out.append((token, price))
    return out

# -------- CoreWeave scraper --------
async def scrape_coreweave_async() -> pd.DataFrame:
    url = "https://www.coreweave.com/pricing"
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url, timeout=90000, wait_until="domcontentloaded")
        # help lazy content load
        for _ in range(3):
            await page.evaluate("window.scrollBy(0, document.body.scrollHeight)")
            await page.wait_for_timeout(800)
        # Wait for any pricing text to appear (best-effort)
        try:
            await page.wait_for_selector("text=/H100|H200/", timeout=5000)
        except Exception:
            pass
        body = await page.inner_text("body")
        await browser.close()

    body = re.sub(r"\s+", " ", body)

    rows = []
    for gpu in ("H100", "H200"):
        for model, price in _nearest_prices(body, gpu, window=240):
            # reasonable hourly range; widen if CoreWeave posts higher tiers
            if not (0.25 <= price <= 25.0):
                continue
            rows.append({
                "provider": "CoreWeave",
                "region": "US",
                "gpu_model": model,
                "type": "On-Demand",
                "duration": "1h",
                "gpu_count": None,
                "price_hourly_usd": price,
                "source_url": url,
                "fetched_at_utc": _now_iso(),
            })

    df = pd.DataFrame(rows, columns=SLIM_COLS)
    if df.empty:
        return _ensure_slim(df)

    # Deduplicate (gpu_model, price)
    df = (df.sort_values(["gpu_model","price_hourly_usd","fetched_at_utc"])
            .drop_duplicates(subset=["gpu_model","price_hourly_usd"], keep="last")
            .reset_index(drop=True))
    return _ensure_slim(df)

# -------- runner (works in scripts & notebooks) --------
def arun(coro):
    try:
        loop = asyncio.get_running_loop()
    except RuntimeError:
        return asyncio.run(coro)
    else:
        import nest_asyncio; nest_asyncio.apply()
        return loop.run_until_complete(coro)

# ------------------------------ RUN --------------------------------
df_coreweave = arun(scrape_coreweave_async())
latest_coreweave = _save_provider(df_coreweave, "coreweave")
display(df_coreweave.head())


[coreweave] snapshot -> docs/data/snapshots/20250904_125500_coreweave.csv
[coreweave] history  -> docs/data/history/coreweave_history.csv
[coreweave] latest   -> docs/data/latest/coreweave_latest.csv


,provider,region,gpu_model,type,duration,gpu_count,price_hourly_usd,source_url,fetched_at_utc
0,CoreWeave,US,H100,On-Demand,1h,None,20.0,https://www.coreweave.com/pricing,2025-09-04 12:55:00


In [17]:
# ================= Paperspace (async) — use your approach, keep correct rows =================
# Output schema (slim): provider, region, gpu_model, type, duration, gpu_count,
#                       price_hourly_usd, source_url, fetched_at_utc

import re, asyncio, pandas as pd, tempfile
from bs4 import BeautifulSoup
from datetime import datetime, timezone
from pathlib import Path
from playwright.async_api import async_playwright

# ---- storage helpers (same as other providers) ----
SLIM_COLS = [
    "provider","region","gpu_model","type","duration","gpu_count",
    "price_hourly_usd","source_url","fetched_at_utc"
]
BASE = Path("docs/data"); HIST_DIR = BASE/"history"; SNAP_DIR = BASE/"snapshots"; LATEST_DIR = BASE/"latest"
for d in (HIST_DIR, SNAP_DIR, LATEST_DIR):
    try: d.mkdir(parents=True, exist_ok=True)
    except Exception: pass

def _now_iso(): return datetime.now(timezone.utc).isoformat(timespec="seconds")

def _ensure_slim(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for c in SLIM_COLS:
        if c not in out.columns: out[c] = None
    out["price_hourly_usd"] = pd.to_numeric(out["price_hourly_usd"], errors="coerce")
    out["fetched_at_utc"] = pd.to_datetime(out["fetched_at_utc"], errors="coerce", utc=True).dt.tz_convert(None)
    return out[SLIM_COLS]

def _safe_to_csv(df: pd.DataFrame, path: Path):
    try:
        path.parent.mkdir(parents=True, exist_ok=True); df.to_csv(path, index=False); return path
    except Exception:
        tmp = Path(tempfile.gettempdir()) / path.name; df.to_csv(tmp, index=False); return tmp

def _save_provider(df: pd.DataFrame, slug: str):
    df = _ensure_slim(df); ts = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M%S")
    snap = _safe_to_csv(df, SNAP_DIR/f"{ts}_{slug}.csv")
    # history
    hist = HIST_DIR/f"{slug}_history.csv"
    if hist.exists():
        old = pd.read_csv(hist, low_memory=False)
        old["fetched_at_utc"] = pd.to_datetime(old["fetched_at_utc"], errors="coerce", utc=True).dt.tz_convert(None)
        all_df = pd.concat([old, df], ignore_index=True)
    else:
        all_df = df.copy()
    all_df = (all_df.dropna(subset=["gpu_model","price_hourly_usd"])
                    .drop_duplicates(subset=["provider","region","gpu_model","type","duration",
                                             "fetched_at_utc","price_hourly_usd"], keep="last")
                    .sort_values("fetched_at_utc"))
    hist = _safe_to_csv(all_df, hist)
    # latest
    key = ["gpu_model","type","region","duration"]
    latest = all_df.sort_values("fetched_at_utc").drop_duplicates(subset=key, keep="last")
    latest_path = _safe_to_csv(latest, LATEST_DIR/f"{slug}_latest.csv")
    print(f"[{slug}] snapshot -> {snap}\n[{slug}] history  -> {hist}\n[{slug}] latest   -> {latest_path}")
    return latest

# ---- strict extractors (but still tolerant to site markup) ----
GPU_PAT = re.compile(r"(H(?:100|200)(?:\s*(?:SXM|PCIE|NVL))?(?:\s*\d{2,3}\s*GB)?)", re.I)
# require an hourly hint somewhere in the same block to avoid platform prices, etc.
PRICE_HOURLY = re.compile(r"\$\s*([0-9]+(?:\.[0-9]+)?)\s*(?:/|\s*(?:per|an)\s*)?(?:GPU\s*/\s*)?(?:h|hr|hour)\b", re.I)

async def scrape_paperspace() -> pd.DataFrame:
    url = "https://www.paperspace.com/pricing"
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url, timeout=90000)               # your simple navigation
        await page.wait_for_timeout(8000)                  # your “just wait a few seconds”
        html = await page.content()
        await browser.close()

    rows = []
    soup = BeautifulSoup(html, "html.parser")
    # scan reasonable blocks; stick to your block-scan approach
    for blk in soup.find_all(["tr","div","section","article","li"], recursive=True):
        txt = blk.get_text(" ", strip=True)
        if not txt: 
            continue
        # must mention H100/H200 AND 'hour' to qualify
        if ("H100" not in txt and "H200" not in txt) or ("hour" not in txt.lower()):
            continue

        # model: first explicit H100/H200-ish token found
        mm = GPU_PAT.search(txt)
        if not mm:
            continue
        model = mm.group(1).upper()

        # price: $… with an hourly hint in the same block
        pm = PRICE_HOURLY.search(txt)
        if not pm:
            continue
        price = float(pm.group(1))
        # sanity band to drop weird captures
        if not (0.2 <= price <= 50.0):
            continue

        rows.append({
            "provider": "Paperspace",
            "region": "Global",
            "gpu_model": model,          # "H100", "H100 PCIE 80GB", etc.
            "type": "On-Demand",
            "duration": "1h",
            "gpu_count": None,
            "price_hourly_usd": price,
            "source_url": url,
            "fetched_at_utc": _now_iso(),
        })

    df = pd.DataFrame(rows, columns=SLIM_COLS)
    if df.empty:
        return _ensure_slim(df)

    # de-dupe (gpu_model, price)
    df = (df.sort_values(["gpu_model","price_hourly_usd","fetched_at_utc"])
            .drop_duplicates(subset=["gpu_model","price_hourly_usd"], keep="last")
            .reset_index(drop=True))
    return _ensure_slim(df)

# ---- runner that works in both scripts & notebooks ----
def arun(coro):
    try:
        loop = asyncio.get_running_loop()
    except RuntimeError:
        return asyncio.run(coro)
    else:
        import nest_asyncio; nest_asyncio.apply()
        return loop.run_until_complete(coro)

# ------------------------------ RUN --------------------------------
df_paperspace = arun(scrape_paperspace())
latest_paperspace = _save_provider(df_paperspace, "paperspace")
display(df_paperspace.head(20))


[paperspace] snapshot -> docs/data/snapshots/20250904_130549_paperspace.csv
[paperspace] history  -> docs/data/history/paperspace_history.csv
[paperspace] latest   -> docs/data/latest/paperspace_latest.csv


,provider,region,gpu_model,type,duration,gpu_count,price_hourly_usd,source_url,fetched_at_utc
0,Paperspace,Global,H100,On-Demand,1h,None,2.24,https://www.paperspace.com/pricing,2025-09-04 13:05:49


In [20]:
# ================= TensorDock H100 (static) — slim schema + per-provider history =================
# Output schema: provider, region, gpu_model, type, duration, gpu_count,
#                price_hourly_usd, source_url, fetched_at_utc
# Py 3.8 compatible

import re, requests, pandas as pd, tempfile
from bs4 import BeautifulSoup
from datetime import datetime, timezone
from pathlib import Path

# -------- pages & patterns (from your code) --------
PAGES = [
    "https://tensordock.com/gpu-h100",
    "https://tensordock.com/cloud-gpus",
    "https://tensordock.com/comparison-gcp",
]
PATTERNS = [
    re.compile(r"H100.*?\$([0-9]+(?:\.[0-9]+)?)\s*/?\s*hr", re.I|re.S),
    re.compile(r"from\s*\$([0-9]+(?:\.[0-9]+)?)\s*/?\s*hr.*?H100", re.I|re.S),
    re.compile(r"\$([0-9]+(?:\.[0-9]+)?)\s*/?\s*hour.*?H100", re.I|re.S),
]
HEADERS = {"User-Agent": "Mozilla/5.0"}

# -------- storage + schema helpers (same as other providers) --------
SLIM_COLS = [
    "provider","region","gpu_model","type","duration","gpu_count",
    "price_hourly_usd","source_url","fetched_at_utc"
]
BASE = Path("docs/data")
HIST_DIR = BASE / "history"
SNAP_DIR = BASE / "snapshots"
LATEST_DIR = BASE / "latest"
for d in (HIST_DIR, SNAP_DIR, LATEST_DIR):
    try:
        d.mkdir(parents=True, exist_ok=True)
    except Exception:
        pass  # fall back handled in _safe_to_csv

def _now_iso() -> str:
    return datetime.now(timezone.utc).isoformat(timespec="seconds")

def _ensure_slim(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for c in SLIM_COLS:
        if c not in out.columns: out[c] = None
    out["price_hourly_usd"] = pd.to_numeric(out["price_hourly_usd"], errors="coerce")
    out["fetched_at_utc"] = pd.to_datetime(out["fetched_at_utc"], errors="coerce", utc=True).dt.tz_convert(None)
    return out[SLIM_COLS]

def _safe_to_csv(df: pd.DataFrame, path: Path):
    try:
        path.parent.mkdir(parents=True, exist_ok=True)
        df.to_csv(path, index=False)
        return path
    except Exception:
        tmp = Path(tempfile.gettempdir()) / path.name
        df.to_csv(tmp, index=False)
        return tmp

def _save_provider(df: pd.DataFrame, provider_slug: str):
    df = _ensure_slim(df)
    ts = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M%S")
    # snapshot
    snap_path = _safe_to_csv(df, SNAP_DIR / f"{ts}_{provider_slug}.csv")
    # history (append + dedupe)
    hist_path = HIST_DIR / f"{provider_slug}_history.csv"
    if hist_path.exists():
        old = pd.read_csv(hist_path, low_memory=False)
        old["fetched_at_utc"] = pd.to_datetime(old["fetched_at_utc"], errors="coerce", utc=True).dt.tz_convert(None)
        all_df = pd.concat([old, df], ignore_index=True)
    else:
        all_df = df.copy()
    all_df = (all_df
              .dropna(subset=["gpu_model","price_hourly_usd"])
              .drop_duplicates(subset=["provider","region","gpu_model","type","duration",
                                       "fetched_at_utc","price_hourly_usd"], keep="last")
              .sort_values("fetched_at_utc"))
    hist_path = _safe_to_csv(all_df, hist_path)
    # latest (newest per gpu/type/region/duration)
    key = ["gpu_model","type","region","duration"]
    latest = all_df.sort_values("fetched_at_utc").drop_duplicates(subset=key, keep="last")
    latest_path = _safe_to_csv(latest, LATEST_DIR / f"{provider_slug}_latest.csv")
    print(f"[{provider_slug}] snapshot -> {snap_path}\n[{provider_slug}] history  -> {hist_path}\n[{provider_slug}] latest   -> {latest_path}")
    return latest

# -------- scraper (uses your logic, mapped to slim schema) --------
def scrape_tensordock_public_h100() -> pd.DataFrame:
    rows = []
    for url in PAGES:
        try:
            r = requests.get(url, headers=HEADERS, timeout=30)
            r.raise_for_status()
            soup = BeautifulSoup(r.text, "html.parser")
            text = soup.get_text(" ", strip=True)
            price = None
            for pat in PATTERNS:
                m = pat.search(text)
                if m:
                    price = float(m.group(1))
                    break
            if price and (0.2 <= price <= 50.0):  # sanity band for $/GPU/hr
                rows.append({
                    "provider": "TensorDock",
                    "region": "Global",
                    "gpu_model": "H100",
                    "type": "On-Demand",
                    "duration": "1h",
                    "gpu_count": 1,
                    "price_hourly_usd": price,
                    "source_url": url,
                    "fetched_at_utc": _now_iso(),
                })
        except Exception as e:
            print(f"[TensorDock] {url} -> {e}")

    if not rows:
        return _ensure_slim(pd.DataFrame(columns=SLIM_COLS))

    # Deduplicate: keep the **lowest** "from" price across pages
    df = pd.DataFrame(rows)
    df = (df.sort_values("price_hourly_usd")
            .drop_duplicates(subset=["provider","gpu_model"], keep="first")
            .reset_index(drop=True))
    return _ensure_slim(df)

# ------------------------------ RUN --------------------------------
df_tensordock = scrape_tensordock_public_h100()
latest_tensordock = _save_provider(df_tensordock, "tensordock")
print(df_tensordock)


[tensordock] snapshot -> docs/data/snapshots/20250904_131631_tensordock.csv
[tensordock] history  -> docs/data/history/tensordock_history.csv
[tensordock] latest   -> docs/data/latest/tensordock_latest.csv
     provider  region gpu_model       type duration  gpu_count  \
0  TensorDock  Global      H100  On-Demand       1h          1   

   price_hourly_usd                       source_url      fetched_at_utc  
0              2.25  https://tensordock.com/gpu-h100 2025-09-04 13:16:31  


In [21]:
# ============== Hydra Host (Brokkr) — slim schema + per-provider history ==============
# Output schema: provider, region, gpu_model, type, duration, gpu_count,
#                price_hourly_usd, source_url, fetched_at_utc
# Py 3.8 compatible

import re, asyncio, pandas as pd, tempfile
from datetime import datetime, timezone
from pathlib import Path
from playwright.async_api import async_playwright

# -------- storage + schema helpers (same pattern as other providers) --------
SLIM_COLS = [
    "provider","region","gpu_model","type","duration","gpu_count",
    "price_hourly_usd","source_url","fetched_at_utc"
]
BASE = Path("docs/data")
HIST_DIR = BASE / "history"
SNAP_DIR = BASE / "snapshots"
LATEST_DIR = BASE / "latest"
for d in (HIST_DIR, SNAP_DIR, LATEST_DIR):
    try: d.mkdir(parents=True, exist_ok=True)
    except Exception: pass

def _now_iso() -> str:
    return datetime.now(timezone.utc).isoformat(timespec="seconds")

def _ensure_slim(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for c in SLIM_COLS:
        if c not in out.columns: out[c] = None
    out["price_hourly_usd"] = pd.to_numeric(out["price_hourly_usd"], errors="coerce")
    out["fetched_at_utc"] = pd.to_datetime(out["fetched_at_utc"], errors="coerce", utc=True).dt.tz_convert(None)
    return out[SLIM_COLS]

def _safe_to_csv(df: pd.DataFrame, path: Path):
    try:
        path.parent.mkdir(parents=True, exist_ok=True)
        df.to_csv(path, index=False)
        return path
    except Exception:
        tmp = Path(tempfile.gettempdir()) / path.name
        df.to_csv(tmp, index=False)
        return tmp

def _save_provider(df: pd.DataFrame, provider_slug: str):
    df = _ensure_slim(df)
    ts = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M%S")
    # snapshot
    snap_path = _safe_to_csv(df, SNAP_DIR / f"{ts}_{provider_slug}.csv")
    # history (append + dedupe)
    hist_path = HIST_DIR / f"{provider_slug}_history.csv"
    if hist_path.exists():
        old = pd.read_csv(hist_path, low_memory=False)
        old["fetched_at_utc"] = pd.to_datetime(old["fetched_at_utc"], errors="coerce", utc=True).dt.tz_convert(None)
        all_df = pd.concat([old, df], ignore_index=True)
    else:
        all_df = df.copy()
    all_df = (
        all_df.dropna(subset=["gpu_model","price_hourly_usd"])
              .drop_duplicates(subset=["provider","region","gpu_model","type","duration",
                                       "fetched_at_utc","price_hourly_usd"], keep="last")
              .sort_values("fetched_at_utc")
    )
    hist_path = _safe_to_csv(all_df, hist_path)
    # latest (newest per gpu/type/region/duration)
    key = ["gpu_model","type","region","duration"]
    latest = all_df.sort_values("fetched_at_utc").drop_duplicates(subset=key, keep="last")
    latest_path = _safe_to_csv(latest, LATEST_DIR / f"{provider_slug}_latest.csv")
    print(f"[{provider_slug}] snapshot -> {snap_path}\n[{provider_slug}] history  -> {hist_path}\n[{provider_slug}] latest   -> {latest_path}")
    return latest

# -------- your Brokkr scraper, tightened to only accept "per card-hour" prices --------
GPU_RE = r"(H100|H200)"
PRICE_PER_CARDHR = r"\$\s*([0-9]+(?:\.[0-9]+)?)\s*(?:per\s*card[-\s]?hour|/card[-\s]?hour)\b"
PATS = [
    re.compile(rf"{GPU_RE}.{{0,220}}?{PRICE_PER_CARDHR}", re.I | re.S),
    re.compile(rf"{PRICE_PER_CARDHR}.{{0,220}}?{GPU_RE}", re.I | re.S),
]

async def scrape_brokkr() -> pd.DataFrame:
    url = "https://brokkr.hydrahost.com/inventory"
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url, timeout=90000, wait_until="domcontentloaded")
        # help hydrate lazy content
        for _ in range(2):
            await page.evaluate("window.scrollBy(0, document.body.scrollHeight)")
            await page.wait_for_timeout(700)
        body_text = await page.inner_text("body")
        await browser.close()

    text = re.sub(r"\s+", " ", body_text)
    rows = []

    for pat in PATS:
        for m in pat.finditer(text):
            # Depending on which pattern matched, group order differs
            groups = m.groups()
            # Normalize extraction: model + price are always present
            if len(groups) == 2:
                # pattern 1: (GPU, price)
                gpu_model, price_str = groups
            elif len(groups) == 3:
                # pattern 2 returns (price, GPU) because of nested groups; pick numeric+gpu
                # groups could be ('12.34', 'H100') or ('12.34', 'card-hour', 'H100') depending on regex engine
                nums = [g for g in groups if g and re.fullmatch(r"[0-9]+(?:\.[0-9]+)?", g)]
                gpus = [g for g in groups if g and re.fullmatch(r"H100|H200", g, flags=re.I)]
                if not nums or not gpus:
                    continue
                price_str, gpu_model = nums[0], gpus[0]
            else:
                # Safe fallback: find first number and first GPU token in the match
                seg = m.group(0)
                pm = re.search(r"[0-9]+(?:\.[0-9]+)?", seg)
                gm = re.search(r"H100|H200", seg, flags=re.I)
                if not (pm and gm):
                    continue
                price_str, gpu_model = pm.group(0), gm.group(0)

            try:
                price = float(price_str)
            except Exception:
                continue

            # sanity band for per-card hour pricing
            if not (0.2 <= price <= 50.0):
                continue

            rows.append({
                "provider": "Hydra Host (Brokkr)",
                "region": "Global",
                "gpu_model": gpu_model.upper(),
                "type": "On-Demand",
                "duration": "1h",
                "gpu_count": 1,
                "price_hourly_usd": price,   # per card-hour = per-GPU hourly
                "source_url": url,
                "fetched_at_utc": _now_iso(),
            })

    df = pd.DataFrame(rows, columns=SLIM_COLS)
    if df.empty:
        return _ensure_slim(df)

    # de-dupe (gpu_model, price)
    df = (df.sort_values(["gpu_model","price_hourly_usd","fetched_at_utc"])
            .drop_duplicates(subset=["gpu_model","price_hourly_usd"], keep="last")
            .reset_index(drop=True))
    return _ensure_slim(df)

# -------- runner that works in both notebooks & scripts --------
def arun(coro):
    try:
        loop = asyncio.get_running_loop()
    except RuntimeError:
        return asyncio.run(coro)
    else:
        import nest_asyncio; nest_asyncio.apply()
        return loop.run_until_complete(coro)

# ------------------------------ RUN --------------------------------
df_brokkr = arun(scrape_brokkr())
latest_brokkr = _save_provider(df_brokkr, "brokkr")
print(df_brokkr)
display(df_brokkr.head())


[brokkr] snapshot -> docs/data/snapshots/20250904_132203_brokkr.csv
[brokkr] history  -> docs/data/history/brokkr_history.csv
[brokkr] latest   -> docs/data/latest/brokkr_latest.csv
              provider  region gpu_model       type duration  gpu_count  \
0  Hydra Host (Brokkr)  Global      H100  On-Demand       1h          1   
1  Hydra Host (Brokkr)  Global      H200  On-Demand       1h          1   

   price_hourly_usd                              source_url  \
0               2.3  https://brokkr.hydrahost.com/inventory   
1               2.5  https://brokkr.hydrahost.com/inventory   

       fetched_at_utc  
0 2025-09-04 13:22:03  
1 2025-09-04 13:22:03  


,provider,region,gpu_model,type,duration,gpu_count,price_hourly_usd,source_url,fetched_at_utc
0,Hydra Host (Brokkr),Global,H100,On-Demand,1h,1,2.3,https://brokkr.hydrahost.com/inventory,2025-09-04 13:22:03
1,Hydra Host (Brokkr),Global,H200,On-Demand,1h,1,2.5,https://brokkr.hydrahost.com/inventory,2025-09-04 13:22:03


In [22]:
# ============ Crusoe Cloud (async) — table scrape → slim schema + history ============

import re, asyncio, pandas as pd, tempfile
from datetime import datetime, timezone
from pathlib import Path
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup

# ---------- slim schema + storage helpers (same as other providers) ----------
SLIM_COLS = [
    "provider","region","gpu_model","type","duration","gpu_count",
    "price_hourly_usd","source_url","fetched_at_utc"
]
BASE = Path("docs/data")
HIST_DIR = BASE / "history"
SNAP_DIR = BASE / "snapshots"
LATEST_DIR = BASE / "latest"
for d in (HIST_DIR, SNAP_DIR, LATEST_DIR):
    try: d.mkdir(parents=True, exist_ok=True)
    except Exception: pass

def _now_iso() -> str:
    return datetime.now(timezone.utc).isoformat(timespec="seconds")

def _ensure_slim(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for c in SLIM_COLS:
        if c not in out.columns: out[c] = None
    out["price_hourly_usd"] = pd.to_numeric(out["price_hourly_usd"], errors="coerce")
    out["fetched_at_utc"] = pd.to_datetime(out["fetched_at_utc"], errors="coerce", utc=True).dt.tz_convert(None)
    return out[SLIM_COLS]

def _safe_to_csv(df: pd.DataFrame, path: Path):
    try:
        path.parent.mkdir(parents=True, exist_ok=True); df.to_csv(path, index=False); return path
    except Exception:
        tmp = Path(tempfile.gettempdir()) / path.name; df.to_csv(tmp, index=False); return tmp

def _save_provider(df: pd.DataFrame, slug: str):
    df = _ensure_slim(df)
    ts = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M%S")
    snap = _safe_to_csv(df, SNAP_DIR / f"{ts}_{slug}.csv")
    # history append+dedupe
    hist = HIST_DIR / f"{slug}_history.csv"
    if hist.exists():
        old = pd.read_csv(hist, low_memory=False)
        old["fetched_at_utc"] = pd.to_datetime(old["fetched_at_utc"], errors="coerce", utc=True).dt.tz_convert(None)
        all_df = pd.concat([old, df], ignore_index=True)
    else:
        all_df = df.copy()
    all_df = (all_df
        .dropna(subset=["gpu_model","price_hourly_usd"])
        .drop_duplicates(subset=["provider","region","gpu_model","type","duration",
                                 "fetched_at_utc","price_hourly_usd"], keep="last")
        .sort_values("fetched_at_utc"))
    hist = _safe_to_csv(all_df, hist)
    # latest per gpu/type/region/duration
    key = ["gpu_model","type","region","duration"]
    latest = all_df.sort_values("fetched_at_utc").drop_duplicates(subset=key, keep="last")
    latest_path = _safe_to_csv(latest, LATEST_DIR / f"{slug}_latest.csv")
    print(f"[{slug}] snapshot -> {snap}\n[{slug}] history  -> {hist}\n[{slug}] latest   -> {latest_path}")
    return latest

# ---------- parsing helpers ----------
PRICE_RE = re.compile(r"\$?\s*([0-9]+(?:\.[0-9]+)?)\s*(?:/|\s*per\s*)?\s*(?:h|hr|hour)\b", re.I)

def _parse_price(cell_text: str):
    if not cell_text: return None
    m = PRICE_RE.search(cell_text.replace(",", ""))
    if not m:
        # fallback: plain $N.NN without explicit /hr
        m2 = re.search(r"\$?\s*([0-9]+(?:\.[0-9]+)?)\b", cell_text.replace(",", ""))
        return float(m2.group(1)) if m2 else None
    return float(m.group(1))

def _is_h_model(text: str) -> bool:
    t = text.upper()
    return ("H100" in t) or ("H200" in t)

def _model_from(text: str) -> str:
    return "H100" if "H100" in text.upper() else "H200"

# ---------- scraper ----------
async def scrape_crusoe_table() -> pd.DataFrame:
    url = "https://www.crusoe.ai/cloud/pricing"
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url, timeout=90000, wait_until="domcontentloaded")
        # help render
        for _ in range(2):
            await page.evaluate("window.scrollBy(0, document.body.scrollHeight)")
            await page.wait_for_timeout(800)
        html = await page.content()
        await browser.close()

    soup = BeautifulSoup(html, "html.parser")
    rows_out = []

    # Find all table rows; filter to those that mention H100/H200
    for tr in soup.find_all("tr"):
        tds = [c.get_text(" ", strip=True) for c in tr.find_all("td")]
        if not tds or not any(_is_h_model(c) for c in tds):
            continue

        model = _model_from(" ".join(tds))

        # Try to map columns conservatively:
        # Common layout: [Model, On-Demand, Spot?, Reserved 6m, Reserved 1y, Reserved 3y, ...]
        # We’ll grab by position if present, else try to read by header alignment.
        on_demand = _parse_price(tds[1]) if len(tds) > 1 else None
        res_6m    = _parse_price(tds[3]) if len(tds) > 3 else None
        res_1y    = _parse_price(tds[4]) if len(tds) > 4 else None
        res_3y    = _parse_price(tds[5]) if len(tds) > 5 else None

        # Build normalised slim rows
        if on_demand is not None:
            rows_out.append({
                "provider": "CrusoeCloud",
                "region": "Global",
                "gpu_model": model,
                "type": "On-Demand",
                "duration": "1h",
                "gpu_count": 1,
                "price_hourly_usd": on_demand,
                "source_url": url,
                "fetched_at_utc": _now_iso(),
            })
        if res_6m is not None:
            rows_out.append({
                "provider": "CrusoeCloud",
                "region": "Global",
                "gpu_model": model,
                "type": "Reserved-6m",
                "duration": "1h",
                "gpu_count": 1,
                "price_hourly_usd": res_6m,
                "source_url": url,
                "fetched_at_utc": _now_iso(),
            })
        if res_1y is not None:
            rows_out.append({
                "provider": "CrusoeCloud",
                "region": "Global",
                "gpu_model": model,
                "type": "Reserved-1y",
                "duration": "1h",
                "gpu_count": 1,
                "price_hourly_usd": res_1y,
                "source_url": url,
                "fetched_at_utc": _now_iso(),
            })
        if res_3y is not None:
            rows_out.append({
                "provider": "CrusoeCloud",
                "region": "Global",
                "gpu_model": model,
                "type": "Reserved-3y",
                "duration": "1h",
                "gpu_count": 1,
                "price_hourly_usd": res_3y,
                "source_url": url,
                "fetched_at_utc": _now_iso(),
            })

    df = pd.DataFrame(rows_out, columns=SLIM_COLS)
    if df.empty:
        return _ensure_slim(df)

    # de-dupe & sanity
    df = (df.sort_values(["gpu_model","type","price_hourly_usd","fetched_at_utc"])
            .drop_duplicates(subset=["gpu_model","type","price_hourly_usd"], keep="last")
            .reset_index(drop=True))
    # plausible hourly band
    df = df[(df["price_hourly_usd"] > 0) & (df["price_hourly_usd"] < 200)]
    return _ensure_slim(df)

# ---------- runner that works in scripts & notebooks ----------
def arun(coro):
    try:
        loop = asyncio.get_running_loop()
    except RuntimeError:
        return asyncio.run(coro)
    else:
        import nest_asyncio; nest_asyncio.apply()
        return loop.run_until_complete(coro)

# ------------------------------ RUN --------------------------------
df_crusoe = arun(scrape_crusoe_table())
latest_crusoe = _save_provider(df_crusoe, "crusoecloud")
print(df_crusoe.head(20))


[crusoecloud] snapshot -> docs/data/snapshots/20250904_132724_crusoecloud.csv
[crusoecloud] history  -> docs/data/history/crusoecloud_history.csv
[crusoecloud] latest   -> docs/data/latest/crusoecloud_latest.csv
      provider  region gpu_model         type duration  gpu_count  \
0  CrusoeCloud  Global      H100    On-Demand       1h          1   
1  CrusoeCloud  Global      H100  Reserved-1y       1h          1   
2  CrusoeCloud  Global      H100  Reserved-3y       1h          1   
3  CrusoeCloud  Global      H100  Reserved-6m       1h          1   
4  CrusoeCloud  Global      H200    On-Demand       1h          1   
5  CrusoeCloud  Global      H200  Reserved-1y       1h          1   
6  CrusoeCloud  Global      H200  Reserved-3y       1h          1   
7  CrusoeCloud  Global      H200  Reserved-6m       1h          1   

   price_hourly_usd                           source_url      fetched_at_utc  
0              3.90  https://www.crusoe.ai/cloud/pricing 2025-09-04 13:27:24  
1       

In [32]:
# OVHcloud H100/H200 — get the *correct per-GPU hourly price* from the public prices table
# - Ties each $…/hour to the same row as H100/H200
# - Extracts the GPU count from the row (1×/2×/4×/8× or “… GPUs”)
# - per_gpu = instance_price / parsed_gpu_count  (NO 8× assumption)
# - Returns both instance price and per-GPU price

import re, requests, pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

def timestamp(): return datetime.utcnow().isoformat()

USD_HOURLY = re.compile(r"\$\s*([0-9]+(?:\.[0-9]+)?)\s*(?:/|\s*(?:per|an)\s*)?(?:h|hr|hour)\b", re.I)

# GPU count detectors (row-level + cell/variant-level)
COUNT_PATS = [
    re.compile(r"(\d+)\s*[×x]\s*(?:NVIDIA\s*)?(H100|H200)\b", re.I),  # "8× H100"
    re.compile(r"(H100|H200)\s*[×x]\s*(\d+)\b", re.I),               # "H100 × 8"
    re.compile(r"(\d+)\s*(?:GPU|GPUs)\b", re.I),                     # "8 GPUs"
    re.compile(r"\b(\d+)\s*[×x]\b", re.I),                           # "4x"
]

def _parse_count(text: str):
    for pat in COUNT_PATS:
        m = pat.search(text)
        if not m: 
            continue
        for g in m.groups():
            if g and g.isdigit():
                n = int(g)
                if 1 <= n <= 16:
                    return n
    return None

def scrape_ovhcloud_correct(url="https://www.ovhcloud.com/en/public-cloud/prices/") -> pd.DataFrame:
    html = requests.get(url, headers={"User-Agent":"Mozilla/5.0"}, timeout=60).text
    soup = BeautifulSoup(html, "html.parser")

    rows = []
    for table in soup.select("table"):
        # headers so we can label which column the number came from
        headers = [th.get_text(" ", strip=True) for th in table.select("thead th")]
        if not headers:
            first = table.find("tr")
            if first:
                headers = [td.get_text(" ", strip=True) for td in first.find_all(["th","td"])]

        body_rows = table.select("tbody tr") or table.select("tr")
        for tr in body_rows:
            tds = tr.find_all("td")
            if not tds: 
                continue
            cells = [td.get_text(" ", strip=True) for td in tds]
            row_txt = " ".join(cells)
            up = row_txt.upper()
            if ("H100" not in up) and ("H200" not in up):
                continue

            model = "H100" if "H100" in up else "H200"
            row_count = _parse_count(row_txt)

            for idx, (td, cell) in enumerate(zip(tds, cells)):
                m = USD_HOURLY.search(cell)
                if not m:
                    continue
                instance_price = float(m.group(1))

                # try counts in cell and header/variant too
                var = headers[idx] if idx < len(headers) and headers else f"col_{idx+1}"
                count = (
                    _parse_count(cell) or
                    _parse_count(var)  or
                    row_count
                )
                if count is None:
                    # if we can't prove node size, skip (prevents wrong divide)
                    continue

                per_gpu = instance_price / count
                if not (0.25 <= per_gpu <= 20.0):
                    continue

                rows.append({
                    "provider": "OVHcloud",
                    "region": "Global",
                    "gpu_model": model,
                    "instance_type": f"public-cloud/{var}",
                    "gpu_count": int(count),
                    "price_hourly_usd_instance": round(instance_price, 4),
                    "price_hourly_usd_per_gpu": round(per_gpu, 4),
                    "price_reserved_usd": None,
                    "reserved_duration": None,
                    "timestamp": timestamp(),
                })

    df = pd.DataFrame(rows)
    if not df.empty:
        df = (df.sort_values(["gpu_model","price_hourly_usd_instance"])
                .drop_duplicates(subset=["gpu_model","instance_type","price_hourly_usd_instance"], keep="last")
                .reset_index(drop=True))
    return df

# Example
df_ovh = scrape_ovhcloud_correct()
print(df_ovh.head(20))

   provider  region gpu_model       instance_type  gpu_count  \
0  OVHcloud  Global      H100  public-cloud/Price          2   
1  OVHcloud  Global      H100  public-cloud/Price          4   

   price_hourly_usd_instance  price_hourly_usd_per_gpu price_reserved_usd  \
0                       5.98                    2.9900               None   
1                      11.97                    2.9925               None   

  reserved_duration                   timestamp  
0              None  2025-09-04T15:18:12.093698  
1              None  2025-09-04T15:18:12.093773  


In [30]:
# === Aggregate all providers → baseline market → score → save derived CSVs ===
# Input: docs/data/latest/*_latest.csv (slim schema)
# Output: docs/data/derived/market_index.csv, provider_scores_latest.csv, price_iq_latest.json

import pandas as pd, numpy as np, json, tempfile
from pathlib import Path
from datetime import datetime, timezone

# Paths
BASE = Path("docs/data")
LATEST_DIR = BASE / "latest"
DERIVED_DIR = BASE / "derived"
for d in (LATEST_DIR, DERIVED_DIR, BASE / "history", BASE / "snapshots"):
    d.mkdir(parents=True, exist_ok=True)

SLIM_COLS = [
    "provider","region","gpu_model","type","duration","gpu_count",
    "price_hourly_usd","source_url","fetched_at_utc"
]

def _now_iso(): 
    return datetime.now(timezone.utc).isoformat(timespec="seconds")

def _safe_to_csv(df: pd.DataFrame, path: Path) -> Path:
    try:
        path.parent.mkdir(parents=True, exist_ok=True)
        df.to_csv(path, index=False)
        return path
    except Exception:
        tmp = Path(tempfile.gettempdir()) / path.name
        df.to_csv(tmp, index=False)
        return tmp

# ---------------- 1) Load & unify (build df FIRST) ----------------
frames = []
for p in sorted(LATEST_DIR.glob("*_latest.csv")):
    try:
        d = pd.read_csv(p)
        for c in SLIM_COLS:
            if c not in d.columns: d[c] = None
        frames.append(d[SLIM_COLS])
    except Exception as e:
        print(f"[WARN] Failed to load {p.name}: {e}")

if not frames:
    raise RuntimeError("No latest provider CSVs found in docs/data/latest/")

raw = pd.concat(frames, ignore_index=True)

# Basic hygiene
raw["gpu_model"] = raw["gpu_model"].astype(str).str.upper().str.strip()
raw["region"]    = raw["region"].astype(str).replace({"nan": None}).fillna("Global").str.strip()
raw["type"]      = raw["type"].astype(str).replace({"nan": None}).fillna("On-Demand").str.strip()
raw["duration"]  = raw["duration"].astype(str).replace({"nan": None}).fillna("1h").str.strip()
raw["gpu_count"] = pd.to_numeric(raw["gpu_count"], errors="coerce")
raw["price_hourly_usd"] = pd.to_numeric(raw["price_hourly_usd"], errors="coerce")
raw["fetched_at_utc"]   = pd.to_datetime(raw["fetched_at_utc"], errors="coerce", utc=True)

# Only H100/H200 and sane prices
df = raw[
    raw["gpu_model"].str.contains(r"\bH100\b|\bH200\b", regex=True, na=False)
].copy()
df = df[(df["price_hourly_usd"] > 0.05) & (df["price_hourly_usd"] < 200)]

# De-dup (latest wins)
dedupe_keys = ["provider","region","gpu_model","type","duration"]
df = (df.sort_values("fetched_at_utc")
        .drop_duplicates(subset=dedupe_keys, keep="last")
        .reset_index(drop=True))

# ---------------- 2) QA (now that df exists) ----------------
qa = df.copy()

# OVH must come from the correct scraper (per-GPU ~ $2–$4 for H100)
qa = qa[~(
    (qa["provider"].eq("OVHcloud")) &
    (qa["gpu_model"].eq("H100")) &
    ((qa["price_hourly_usd"] < 2.0) | (qa["price_hourly_usd"] > 4.0))
)]

# Shadeform: drop obvious marketing/outlier rows
qa = qa[~(
    (qa["provider"].eq("Shadeform")) &
    (qa["price_hourly_usd"] > 6.0)
)]

if qa.empty:
    raise RuntimeError("No rows after QA; check scrapers or relax QA filters.")

# ---------------- 3) Market baselines from On-Demand only ----------------
GROUP = ["gpu_model", "region"]
od = qa[qa["type"].eq("On-Demand")].copy()
if od.empty:
    raise RuntimeError("No On-Demand rows to build market baselines.")

g = od.groupby(GROUP)["price_hourly_usd"]
market = pd.DataFrame({
    "market_count": g.size(),
    "market_median": g.median(),
    "market_mean": g.mean(),
    "market_p25": g.quantile(0.25),
    "market_p75": g.quantile(0.75),
}).reset_index()
market["market_iqr"] = market["market_p75"] - market["market_p25"]
market["asof_utc"] = _now_iso()

# ---------------- 4) Score all rows vs market ----------------
scored = qa.merge(market, on=GROUP, how="left")
scored["premium_vs_median"] = (scored["price_hourly_usd"] - scored["market_median"]) / scored["market_median"]
iqr = scored["market_iqr"].replace(0, np.nan)
scored["is_outlier"] = (
    (scored["price_hourly_usd"] < scored["market_p25"] - 1.5*iqr) |
    (scored["price_hourly_usd"] > scored["market_p75"] + 1.5*iqr)
)
# Price-IQ score (100 = at median; higher = cheaper)
scored["price_score"] = (scored["market_median"] / scored["price_hourly_usd"] * 100).clip(50, 150).round(1)

# ---------------- 5) Save derived artifacts ----------------
market_path = _safe_to_csv(market, DERIVED_DIR / "market_index.csv")
scores_path = _safe_to_csv(scored, DERIVED_DIR / "provider_scores_latest.csv")

# Lightweight JSON for dashboard (On-Demand leaderboard)
leaderboard = (scored[scored["type"].eq("On-Demand")]
               .sort_values(["gpu_model","region","price_score"], ascending=[True, True, False]))
(lb := leaderboard[["provider","region","gpu_model","type","price_hourly_usd","price_score","premium_vs_median","source_url"]].copy()) \
    .assign(premium_vs_median=lambda x: x["premium_vs_median"].round(4)) \
    .to_json(DERIVED_DIR / "price_iq_latest.json", orient="records", indent=2)

print("Derived saved:\n- ", market_path, "\n- ", scores_path, "\n- ", DERIVED_DIR / "price_iq_latest.json")
print(leaderboard.head(12)[["provider","region","gpu_model","type","price_hourly_usd","price_score"]].to_string(index=False))


Derived saved:
-  docs/data/derived/market_index.csv 
-  docs/data/derived/provider_scores_latest.csv 
-  docs/data/derived/price_iq_latest.json
            provider  region gpu_model       type  price_hourly_usd  price_score
             Vast.ai  Global      H100  On-Demand              1.25        150.0
              Nebius  Global      H100  On-Demand              2.00        112.2
          Paperspace  Global      H100  On-Demand              2.24        100.2
          TensorDock  Global      H100  On-Demand              2.25         99.8
 Hydra Host (Brokkr)  Global      H100  On-Demand              2.30         97.6
         CrusoeCloud  Global      H100  On-Demand              3.90         57.6
         VoltagePark      US      H100  On-Demand              1.99        150.0
           CoreWeave      US      H100  On-Demand             20.00         55.0
              Nebius  Global      H200  On-Demand              2.30        107.6
           Shadeform  Global      H200  On-De

In [2]:
# ====================== PRICE IQ FORMULA (Explainable, Weighted) ======================
# Input: docs/data/derived/provider_scores_latest.csv (created by the aggregator)
# Output: docs/data/derived/price_iq_explainable.csv  +  docs/data/derived/price_iq_config.json
from pathlib import Path
from datetime import datetime, timezone
import pandas as pd, numpy as np, json, tempfile, re

DERIVED_DIR = Path("docs/data/derived")
DERIVED_DIR.mkdir(parents=True, exist_ok=True)

# ---- 1) Load scored rows (has market_* columns already) ----
scores_path = DERIVED_DIR / "provider_scores_latest.csv"
if not scores_path.exists():
    raise SystemExit(f"Missing {scores_path}. Run the aggregator cell first.")

df = pd.read_csv(scores_path)

# ---- 2) Minimal features we need ----
MUST = ["provider","region","gpu_model","type","price_hourly_usd","market_median","source_url"]
for c in MUST:
    if c not in df.columns:
        raise SystemExit(f"Missing column in provider_scores_latest.csv: {c}")

# Keep just the latest per (provider, region, model, type)
if "fetched_at_utc" in df.columns:
    df = df.sort_values("fetched_at_utc")
df = df.drop_duplicates(subset=["provider","region","gpu_model","type"], keep="last")

# ---- 3) Optional signals (perf/avail/sla) — default to neutral 1.0 ----
def _infer_perf(name:str) -> float:
    n = (str(name) or "").upper()
    if "SXM" in n: return 1.00
    if "NVL" in n: return 1.02
    if "PCIE" in n: return 0.95
    return 1.00

def _infer_avail(_): return 1.00
def _infer_sla(_):   return 1.00

df["perf"]  = df["gpu_model"].map(_infer_perf)
df["avail"] = df["provider"].map(_infer_avail)
df["sla"]   = df["provider"].map(_infer_sla)

# ---- 4) Price component (primary signal) ----
# price_component = market_median / price_hourly_usd
#   1.0 = at market median, >1 cheaper than median, <1 pricier than median
df["price_component"] = (df["market_median"] / df["price_hourly_usd"]).clip(0.01, 10.0)

# ---- 5) Weights (edit here or surface in UI later) ----
WEIGHTS = {
    "price": 1.00,   # strongest driver (v1: price-only)
    "perf":  0.00,   # enable (e.g., 0.10) to reward SXM/NVL over PCIe (heuristic)
    "avail": 0.00,   # future capacity/queue signal
    "sla":   0.00,   # future reliability/SLA signal
}
w_sum = sum(WEIGHTS.values()) or 1.0
W = {k: v / w_sum for k, v in WEIGHTS.items()}

# ---- 6) Price-IQ score ----
# IQ = 100 * ( w_price * (M/P) + w_perf*perf + w_avail*avail + w_sla*sla ), clipped to [50,150]
raw = (W["price"]*df["price_component"] +
       W.get("perf",0)*df["perf"] +
       W.get("avail",0)*df["avail"] +
       W.get("sla",0)*df["sla"])
df["price_iq"] = (100 * raw).clip(50, 150).round(1)

# Also expose premium vs median for UI copy
df["premium_vs_median"] = ((df["price_hourly_usd"] - df["market_median"]) / df["market_median"]).round(4)

# ---- 7) Save explainable table + config ----
keep = ["provider","region","gpu_model","type","price_hourly_usd","market_median",
        "price_component","perf","avail","sla","price_iq","premium_vs_median","source_url"]
explain = (df[keep]
           .sort_values(["gpu_model","region","price_iq"], ascending=[True, True, False])
           .reset_index(drop=True))

out_csv = DERIVED_DIR / "price_iq_explainable.csv"
explain.to_csv(out_csv, index=False)

cfg = {
    "generated_at_utc": datetime.now(timezone.utc).isoformat(timespec="seconds"),
    "weights": W,
    "formula": "IQ = 100 * (w_price*(M/P) + w_perf*perf + w_avail*avail + w_sla*sla); clipped [50,150]",
    "notes": {
        "M": "market_median for same gpu_model & region (On-Demand baseline)",
        "P": "provider per-GPU hourly price",
        "perf/avail/sla": "neutral 1.0 by default; plug real signals when available"
    }
}
with open(DERIVED_DIR / "price_iq_config.json","w") as f:
    json.dump(cfg, f, indent=2)

print("Saved:", out_csv, "and price_iq_config.json")
print(explain.head(12).to_string(index=False))
# ====================== /PRICE IQ FORMULA ======================


Saved: docs/data/derived/price_iq_explainable.csv and price_iq_config.json
            provider  region gpu_model         type  price_hourly_usd  market_median  price_component  perf  avail  sla  price_iq  premium_vs_median                                                    source_url
             Vast.ai  Global      H100    On-Demand              1.25          2.245         1.796000   1.0    1.0  1.0     150.0            -0.4432                            https://vast.ai/products/gpu-cloud
              Nebius  Global      H100    On-Demand              2.00          2.245         1.122500   1.0    1.0  1.0     112.2            -0.1091                                     https://nebius.com/prices
          Paperspace  Global      H100    On-Demand              2.24          2.245         1.002232   1.0    1.0  1.0     100.2            -0.0022                            https://www.paperspace.com/pricing
          TensorDock  Global      H100    On-Demand              2.25          2.

In [14]:
# ========================== ROI / EARNINGS CALCULATOR (fixed) ==========================
from pathlib import Path
import pandas as pd, numpy as np, json

DERIVED = Path("docs/data/derived"); DERIVED.mkdir(parents=True, exist_ok=True)
scores_path = DERIVED / "provider_scores_latest.csv"
if not scores_path.exists():
    raise SystemExit("Missing provider_scores_latest.csv. Run the aggregator first.")
scores = pd.read_csv(scores_path)

def roi_compare(
    gpu_model="H100",
    region=None,                     # e.g. "Global"; None = all regions
    type_filter=("On-Demand",),      # include reserved types if desired
    gpus=8,
    hours=1000,
    utilization=0.9,                 # 0..1
    extra_fees_perc=0.00,            # platform/markup %, e.g. 0.05 = +5%
    egress_gb=0.0,
    egress_cost_per_gb=0.0,
    storage_cost=0.0,                # flat $
    revenue_per_unit=None,           # optional revenue model
    units_per_gpu_hour=None          # optional throughput
) -> pd.DataFrame:
    df = scores.copy()
    df = df[df["gpu_model"].str.upper().eq(gpu_model.upper())]
    if region is not None:
        df = df[df["region"].eq(region)]
    if type_filter:
        df = df[df["type"].isin(type_filter)]
    df = df.dropna(subset=["price_hourly_usd","market_median"])

    if df.empty:
        cols = ["gpu_model","provider","region","type","price_hourly_usd","market_median",
                "gpu_hours_eff","infra_cost","extra_fees","egress_cost","storage_cost",
                "total_cost","delta_vs_cheapest","savings_vs_median",
                "expected_revenue","gross_margin","roi_pct","source_url"]
        return pd.DataFrame(columns=cols)

    # Effective GPU-hours actually used
    gpu_hours_eff = float(gpus) * float(hours) * max(0.0, min(1.0, float(utilization)))
    df = df.assign(gpu_hours_eff=gpu_hours_eff)

    # Cost breakdown
    df["infra_cost"]   = (df["price_hourly_usd"] * df["gpu_hours_eff"]).round(2)
    df["extra_fees"]   = (df["infra_cost"] * float(extra_fees_perc)).round(2)
    df["egress_cost"]  = float(egress_gb) * float(egress_cost_per_gb)
    df["storage_cost"] = float(storage_cost)
    df["total_cost"]   = (df["infra_cost"] + df["extra_fees"] + df["egress_cost"] + df["storage_cost"]).round(2)

    # Benchmarks
    cheapest_total = df["total_cost"].min()
    df["delta_vs_cheapest"] = (df["total_cost"] - cheapest_total).round(2)      # >= 0
    df["median_cost_same_hours"] = (df["market_median"] * df["gpu_hours_eff"]).round(2)
    df["savings_vs_median"] = (df["median_cost_same_hours"] - df["total_cost"]).round(2)

    # Optional earnings / ROI
    if (revenue_per_unit is not None) and (units_per_gpu_hour is not None):
        exp_rev = float(revenue_per_unit) * float(units_per_gpu_hour) * df["gpu_hours_eff"]
        df["expected_revenue"] = exp_rev.round(2)
        df["gross_margin"] = (df["expected_revenue"] - df["total_cost"]).round(2)
        df["roi_pct"] = np.where(df["total_cost"] > 0,
                                 (df["gross_margin"] / df["total_cost"]) * 100, np.nan).round(2)
    else:
        df["expected_revenue"] = np.nan
        df["gross_margin"] = np.nan
        df["roi_pct"] = np.nan

    # keep gpu_model in output
    keep = ["gpu_model","provider","region","type","price_hourly_usd","market_median",
            "gpu_hours_eff","infra_cost","extra_fees","egress_cost","storage_cost",
            "total_cost","delta_vs_cheapest","savings_vs_median",
            "expected_revenue","gross_margin","roi_pct","source_url"]

    out = (df[keep]
           .sort_values(["gpu_model","total_cost","provider"], ascending=[True, True, True])
           .reset_index(drop=True))
    return out

# Example run (edit as needed)
roi_table = roi_compare(
    gpu_model="H100", region=None, type_filter=("On-Demand",),
    gpus=8, hours=1000, utilization=0.90,
    extra_fees_perc=0.00, egress_gb=0, egress_cost_per_gb=0.0, storage_cost=0.0,
    # revenue_per_unit=0.002, units_per_gpu_hour=50000
)

# Save once
csv_out = DERIVED / "roi_comparison_latest.csv"
json_out = DERIVED / "roi_comparison_latest.json"
roi_table.to_csv(csv_out, index=False)
with open(json_out, "w") as f:
    json.dump(roi_table.to_dict(orient="records"), f, indent=2)
print("Saved ->", csv_out, "and", json_out)

roi_table.head(10)
# ======================== /ROI / EARNINGS CALCULATOR (fixed) ==========================


Saved -> docs/data/derived/roi_comparison_latest.csv and docs/data/derived/roi_comparison_latest.json


,gpu_model,provider,region,type,price_hourly_usd,market_median,gpu_hours_eff,infra_cost,extra_fees,egress_cost,storage_cost,total_cost,delta_vs_cheapest,savings_vs_median,expected_revenue,gross_margin,roi_pct,source_url
0,H100,Vast.ai,Global,On-Demand,1.25,2.245,7200.0,9000.0,0.0,0.0,0.0,9000.0,0.0,7164.0,NaN,NaN,NaN,https://vast.ai/products/gpu-cloud
1,H100,VoltagePark,US,On-Demand,1.99,10.995,7200.0,14328.0,0.0,0.0,0.0,14328.0,5328.0,64836.0,NaN,NaN,NaN,https://dashboard.voltagepark.com/order/config...
2,H100,Nebius,Global,On-Demand,2.00,2.245,7200.0,14400.0,0.0,0.0,0.0,14400.0,5400.0,1764.0,NaN,NaN,NaN,https://nebius.com/prices
3,H100,Paperspace,Global,On-Demand,2.24,2.245,7200.0,16128.0,0.0,0.0,0.0,16128.0,7128.0,36.0,NaN,NaN,NaN,https://www.paperspace.com/pricing
4,H100,TensorDock,Global,On-Demand,2.25,2.245,7200.0,16200.0,0.0,0.0,0.0,16200.0,7200.0,-36.0,NaN,NaN,NaN,https://tensordock.com/gpu-h100
5,H100,Hydra Host (Brokkr),Global,On-Demand,2.30,2.245,7200.0,16560.0,0.0,0.0,0.0,16560.0,7560.0,-396.0,NaN,NaN,NaN,https://brokkr.hydrahost.com/inventory
6,H100,CrusoeCloud,Global,On-Demand,3.90,2.245,7200.0,28080.0,0.0,0.0,0.0,28080.0,19080.0,-11916.0,NaN,NaN,NaN,https://www.crusoe.ai/cloud/pricing
7,H100,CoreWeave,US,On-Demand,20.00,10.995,7200.0,144000.0,0.0,0.0,0.0,144000.0,135000.0,-64836.0,NaN,NaN,NaN,https://www.coreweave.com/pricing


In [15]:
# Include all regions + compute ROI
roi_table = roi_compare(
    gpu_model="H100",
    region=None,                       # <- include Global, US, EU, etc.
    type_filter=("On-Demand",),        # add reserved: ("On-Demand","Reserved-6m","Reserved-1y","Reserved-3y")
    gpus=8, hours=1000, utilization=0.90,
    extra_fees_perc=0.00, egress_gb=0, egress_cost_per_gb=0.0, storage_cost=0.0,
    revenue_per_unit=0.002,            # e.g., $/unit (token/frame/etc.)
    units_per_gpu_hour=50000           # e.g., units produced per GPU-hour
)
roi_table.head(15)


,gpu_model,provider,region,type,price_hourly_usd,market_median,gpu_hours_eff,infra_cost,extra_fees,egress_cost,storage_cost,total_cost,delta_vs_cheapest,savings_vs_median,expected_revenue,gross_margin,roi_pct,source_url
0,H100,Vast.ai,Global,On-Demand,1.25,2.245,7200.0,9000.0,0.0,0.0,0.0,9000.0,0.0,7164.0,720000.0,711000.0,7900.00,https://vast.ai/products/gpu-cloud
1,H100,VoltagePark,US,On-Demand,1.99,10.995,7200.0,14328.0,0.0,0.0,0.0,14328.0,5328.0,64836.0,720000.0,705672.0,4925.13,https://dashboard.voltagepark.com/order/config...
2,H100,Nebius,Global,On-Demand,2.00,2.245,7200.0,14400.0,0.0,0.0,0.0,14400.0,5400.0,1764.0,720000.0,705600.0,4900.00,https://nebius.com/prices
3,H100,Paperspace,Global,On-Demand,2.24,2.245,7200.0,16128.0,0.0,0.0,0.0,16128.0,7128.0,36.0,720000.0,703872.0,4364.29,https://www.paperspace.com/pricing
4,H100,TensorDock,Global,On-Demand,2.25,2.245,7200.0,16200.0,0.0,0.0,0.0,16200.0,7200.0,-36.0,720000.0,703800.0,4344.44,https://tensordock.com/gpu-h100
5,H100,Hydra Host (Brokkr),Global,On-Demand,2.30,2.245,7200.0,16560.0,0.0,0.0,0.0,16560.0,7560.0,-396.0,720000.0,703440.0,4247.83,https://brokkr.hydrahost.com/inventory
6,H100,CrusoeCloud,Global,On-Demand,3.90,2.245,7200.0,28080.0,0.0,0.0,0.0,28080.0,19080.0,-11916.0,720000.0,691920.0,2464.10,https://www.crusoe.ai/cloud/pricing
7,H100,CoreWeave,US,On-Demand,20.00,10.995,7200.0,144000.0,0.0,0.0,0.0,144000.0,135000.0,-64836.0,720000.0,576000.0,400.00,https://www.coreweave.com/pricing


In [16]:
# FX normalize -> add "price_usd" everywhere
import pandas as pd, json, math
from pathlib import Path
DERIVED = Path("docs/data/derived")

# 1) tiny FX map (extend via ECB later)
FX = {"USD":1.0,"EUR":1.08,"GBP":1.29}  # $/unit

scores = pd.read_csv(DERIVED/"provider_scores_latest.csv")
if "currency" not in scores.columns: scores["currency"]="USD"
scores["price_usd"] = scores.apply(lambda r: (r["price_hourly_usd"] / FX.get(r["currency"],1.0))
                                   if not math.isnan(r["price_hourly_usd"]) else None, axis=1)
scores.to_csv(DERIVED/"provider_scores_latest.csv", index=False)
print("FX-normalized -> provider_scores_latest.csv (price_usd)")


FX-normalized -> provider_scores_latest.csv (price_usd)


In [17]:
# weights.json drives the formula; edit without touching code
from pathlib import Path; import json, pandas as pd, numpy as np, datetime as dt
DERIVED = Path("docs/data/derived")
cfg_path = DERIVED/"price_iq_weights.json"
if not cfg_path.exists():
    cfg_path.write_text(json.dumps({"price":1.0,"perf":0.1,"avail":0.0,"sla":0.0}, indent=2))
W = json.loads(cfg_path.read_text())
scores = pd.read_csv(DERIVED/"provider_scores_latest.csv")

# perf heuristic (SXM≈1.00, NVL≈1.02, PCIe≈0.95)
def perf(x:str)->float:
    s=(x or "").upper(); 
    return 1.02 if "NVL" in s else 1.00 if "SXM" in s else 0.95 if "PCIE" in s else 1.0

scores["perf"]  = scores["gpu_model"].map(perf)
scores["avail"] = 1.0
scores["sla"]   = 1.0
scores["price_component"] = (scores["market_median"]/scores["price_hourly_usd"]).clip(0.01,10)

totw = sum(W.values()) or 1.0; Wn={k:v/totw for k,v in W.items()}
raw = (Wn["price"]*scores["price_component"] + Wn["perf"]*scores["perf"] +
       Wn.get("avail",0)*scores["avail"] + Wn.get("sla",0)*scores["sla"])
scores["price_iq"] = (100*raw).clip(50,150).round(1)
scores.to_csv(DERIVED/"price_iq_explainable.csv", index=False)
print("Saved -> price_iq_explainable.csv  (weights:", Wn, ")")


Saved -> price_iq_explainable.csv  (weights: {'price': 0.9090909090909091, 'perf': 0.09090909090909091, 'avail': 0.0, 'sla': 0.0} )


In [9]:
# anomaly scan: day-over-day delta + empty checks
from pathlib import Path; import pandas as pd, json
DERIVED, HISTORY = Path("docs/data/derived"), Path("docs/data/history")
scores = pd.read_csv(DERIVED/"provider_scores_latest.csv")
alerts=[]

# empty checks
for f in ["market_index.csv","provider_scores_latest.csv","price_iq_explainable.csv"]:
    p=DERIVED/f
    if not p.exists() or p.stat().st_size<200: alerts.append(f"{f} missing/empty")

# day-over-day price jumps > 20%
for hist in HISTORY.glob("*_history.csv"):
    try:
        h=pd.read_csv(hist)
        last=h.sort_values("fetched_at_utc").groupby(["provider","gpu_model"]).tail(1)
        cur=scores.groupby(["provider","gpu_model"])["price_hourly_usd"].mean()
        merged=last.set_index(["provider","gpu_model"])["price_hourly_usd"].to_frame("prev").join(cur.to_frame("now"))
        jump=((merged["now"]-merged["prev"]).abs()/merged["prev"]).fillna(0)
        bad=jump[jump>0.2]
        for (prov,model),v in bad.items():
            alerts.append(f"{prov} {model} price jump {v:.0%}")
    except Exception: pass

Path("docs/data/derived/alerts.json").write_text(json.dumps(alerts, indent=2))
print("Alerts:", alerts[:5])


Alerts: []


In [18]:
# break-even price + sensitivity table
import pandas as pd, numpy as np
from pathlib import Path
DERIVED=Path("docs/data/derived")
scores=pd.read_csv(DERIVED/"provider_scores_latest.csv")

def roi_grid(gpu_model="H100", gpus=8, hours_list=(200,500,1000), util_list=(0.5,0.8,0.95)):
    df=scores.query("gpu_model == @gpu_model and type == 'On-Demand'").copy()
    out=[]
    for h in hours_list:
        for u in util_list:
            eff=gpus*h*u
            cost=(df["price_hourly_usd"]*eff)
            be_price=(df["market_median"])  # median is neutral; BE is just M for savings=0
            out.append(pd.DataFrame({
                "provider":df["provider"],"region":df["region"],
                "hours":h,"util":u,"price":df["price_hourly_usd"].round(3),
                "total_cost":cost.round(2),
                "break_even_price":be_price.round(3),
                "savings_vs_median":(df["market_median"]*eff - cost).round(2)
            }))
    grid=pd.concat(out).sort_values(["hours","util","total_cost"])
    grid.to_csv(DERIVED/"roi_scenarios.csv", index=False)
    print("Saved -> roi_scenarios.csv")
    return grid

roi_grid()


Saved -> roi_scenarios.csv


,provider,region,hours,util,price,total_cost,break_even_price,savings_vs_median
4,Vast.ai,Global,200,0.50,1.25,1000.0,2.245,796.0
3,VoltagePark,US,200,0.50,1.99,1592.0,10.995,7204.0
1,Nebius,Global,200,0.50,2.00,1600.0,2.245,196.0
6,Paperspace,Global,200,0.50,2.24,1792.0,2.245,4.0
8,TensorDock,Global,200,0.50,2.25,1800.0,2.245,-4.0
...,...,...,...,...,...,...,...,...
6,Paperspace,Global,1000,0.95,2.24,17024.0,2.245,38.0
8,TensorDock,Global,1000,0.95,2.25,17100.0,2.245,-38.0
9,Hydra Host (Brokkr),Global,1000,0.95,2.30,17480.0,2.245,-418.0
16,CrusoeCloud,Global,1000,0.95,3.90,29640.0,2.245,-12578.0


In [19]:
# --- Sanity check: derived artifacts exist & look sane ---
from pathlib import Path
import pandas as pd, json

DER = Path("docs/data/derived")

need_csv = [
    "provider_scores_latest.csv",
    "market_index.csv",
    "price_iq_explainable.csv",
    "roi_comparison_latest.csv",
]
for f in need_csv:
    p = DER / f
    assert p.exists() and p.stat().st_size > 200, f"Missing/empty: {p}"

scores = pd.read_csv(DER/"provider_scores_latest.csv")
market = pd.read_csv(DER/"market_index.csv")
iq     = pd.read_csv(DER/"price_iq_explainable.csv")
roi    = pd.read_csv(DER/"roi_comparison_latest.csv")

print("rows — scores:", len(scores), "market:", len(market), "iq:", len(iq), "roi:", len(roi))
assert scores["price_hourly_usd"].gt(0).any(), "prices look empty"
if "price_score" in scores.columns:
    assert scores["price_score"].between(50,150).any(), "price_score missing/out of range"
assert iq["price_iq"].between(50,150).any(), "price_iq missing/out of range"

cfg = DER/"price_iq_config.json"
print("found config:", cfg.exists(), cfg.read_text()[:120] + "..." if cfg.exists() else "no config")

display(iq.head(5))
display(roi.head(5))


rows — scores: 19 market: 4 iq: 19 roi: 8
found config: True {
  "generated_at_utc": "2025-09-08T10:14:01+00:00",
  "weights": {
    "price": 1.0,
    "perf": 0.0,
    "avail": 0.0,...


,provider,region,gpu_model,type,duration,gpu_count,price_hourly_usd,source_url,fetched_at_utc,market_count,...,premium_vs_median,is_outlier,price_score,currency,price_usd,perf,avail,sla,price_component,price_iq
0,Nebius,US,H200,On-Demand,1h,NaN,2.30,https://nebius.com/h200,2025-09-04 11:40:47+00:00,1,...,0.000000,False,100.0,USD,2.30,1.0,1.0,1.0,1.000000,100.0
1,Nebius,Global,H100,On-Demand,1h,NaN,2.00,https://nebius.com/prices,2025-09-04 11:42:58.443649+00:00,6,...,-0.109131,False,112.2,USD,2.00,1.0,1.0,1.0,1.122500,111.1
2,Nebius,Global,H200,On-Demand,1h,NaN,2.30,https://nebius.com/prices,2025-09-04 11:42:58.443649+00:00,4,...,-0.070707,False,107.6,USD,2.30,1.0,1.0,1.0,1.076087,106.9
3,VoltagePark,US,H100,On-Demand,1h,NaN,1.99,https://dashboard.voltagepark.com/order/config...,2025-09-04 12:36:26+00:00,2,...,-0.819009,False,150.0,USD,1.99,1.0,1.0,1.0,5.525126,150.0
4,Vast.ai,Global,H100,On-Demand,1h,NaN,1.25,https://vast.ai/products/gpu-cloud,2025-09-04 12:42:14+00:00,6,...,-0.443207,True,150.0,USD,1.25,1.0,1.0,1.0,1.796000,150.0


,gpu_model,provider,region,type,price_hourly_usd,market_median,gpu_hours_eff,infra_cost,extra_fees,egress_cost,storage_cost,total_cost,delta_vs_cheapest,savings_vs_median,expected_revenue,gross_margin,roi_pct,source_url
0,H100,Vast.ai,Global,On-Demand,1.25,2.245,7200.0,9000.0,0.0,0.0,0.0,9000.0,0.0,7164.0,NaN,NaN,NaN,https://vast.ai/products/gpu-cloud
1,H100,VoltagePark,US,On-Demand,1.99,10.995,7200.0,14328.0,0.0,0.0,0.0,14328.0,5328.0,64836.0,NaN,NaN,NaN,https://dashboard.voltagepark.com/order/config...
2,H100,Nebius,Global,On-Demand,2.00,2.245,7200.0,14400.0,0.0,0.0,0.0,14400.0,5400.0,1764.0,NaN,NaN,NaN,https://nebius.com/prices
3,H100,Paperspace,Global,On-Demand,2.24,2.245,7200.0,16128.0,0.0,0.0,0.0,16128.0,7128.0,36.0,NaN,NaN,NaN,https://www.paperspace.com/pricing
4,H100,TensorDock,Global,On-Demand,2.25,2.245,7200.0,16200.0,0.0,0.0,0.0,16200.0,7200.0,-36.0,NaN,NaN,NaN,https://tensordock.com/gpu-h100


In [12]:
# --- Quick previews: Top Price-IQ and Cheapest ROI ---
import pandas as pd
from pathlib import Path

DER = Path("docs/data/derived")
iq = pd.read_csv(DER/"price_iq_explainable.csv")
roi = pd.read_csv(DER/"roi_comparison_latest.csv")

print("\nTop by Price-IQ (per model):")
for model in sorted(iq["gpu_model"].dropna().unique()):
    print(f"\n== {model} ==")
    display(iq[iq["gpu_model"]==model]
            .sort_values("price_iq", ascending=False)
            .loc[:, ["provider","region","type","price_hourly_usd","market_median","price_iq","source_url"]]
            .head(8))

print("\nCheapest total cost (ROI table):")
display(roi.sort_values(["total_cost","provider"]).head(10)
        [["provider","region","type","price_hourly_usd","total_cost","savings_vs_median","source_url"]])



Top by Price-IQ (per model):

== H100 ==


,provider,region,type,price_hourly_usd,market_median,price_iq,source_url
3,VoltagePark,US,On-Demand,1.99,10.995,150.0,https://dashboard.voltagepark.com/order/config...
4,Vast.ai,Global,On-Demand,1.25,2.245,150.0,https://vast.ai/products/gpu-cloud
1,Nebius,Global,On-Demand,2.00,2.245,111.1,https://nebius.com/prices
6,Paperspace,Global,On-Demand,2.24,2.245,100.2,https://www.paperspace.com/pricing
8,TensorDock,Global,On-Demand,2.25,2.245,99.8,https://tensordock.com/gpu-h100
9,Hydra Host (Brokkr),Global,On-Demand,2.30,2.245,97.8,https://brokkr.hydrahost.com/inventory
14,CrusoeCloud,Global,Reserved-3y,2.54,2.245,89.4,https://www.crusoe.ai/cloud/pricing
15,CrusoeCloud,Global,Reserved-1y,2.93,2.245,78.7,https://www.crusoe.ai/cloud/pricing



== H200 ==


,provider,region,type,price_hourly_usd,market_median,price_iq,source_url
2,Nebius,Global,On-Demand,2.30,2.475,106.9,https://nebius.com/prices
5,Shadeform,Global,On-Demand,2.45,2.475,100.9,https://www.shadeform.ai/
0,Nebius,US,On-Demand,2.30,2.300,100.0,https://nebius.com/h200
10,Hydra Host (Brokkr),Global,On-Demand,2.50,2.475,99.1,https://brokkr.hydrahost.com/inventory
18,CrusoeCloud,Global,Reserved-3y,2.79,2.475,89.7,https://www.crusoe.ai/cloud/pricing
11,CrusoeCloud,Global,Reserved-1y,3.22,2.475,79.0,https://www.crusoe.ai/cloud/pricing
17,CrusoeCloud,Global,Reserved-6m,3.43,2.475,74.7,https://www.crusoe.ai/cloud/pricing
12,CrusoeCloud,Global,On-Demand,4.29,2.475,61.5,https://www.crusoe.ai/cloud/pricing



Cheapest total cost (ROI table):


,provider,region,type,price_hourly_usd,total_cost,savings_vs_median,source_url
0,Vast.ai,Global,On-Demand,1.25,9000.0,7164.0,https://vast.ai/products/gpu-cloud
1,Nebius,Global,On-Demand,2.00,14400.0,1764.0,https://nebius.com/prices
2,Paperspace,Global,On-Demand,2.24,16128.0,36.0,https://www.paperspace.com/pricing
3,TensorDock,Global,On-Demand,2.25,16200.0,-36.0,https://tensordock.com/gpu-h100
4,Hydra Host (Brokkr),Global,On-Demand,2.30,16560.0,-396.0,https://brokkr.hydrahost.com/inventory
5,CrusoeCloud,Global,On-Demand,3.90,28080.0,-11916.0,https://www.crusoe.ai/cloud/pricing


In [20]:
# ================= BACKTEST: build daily panel & recompute daily Price-IQ =================
from pathlib import Path
import pandas as pd, numpy as np
from datetime import timezone

HIST = Path("docs/data/history")
DER  = Path("docs/data/derived"); DER.mkdir(parents=True, exist_ok=True)

# Load all *_history.csv that follow the slim schema (+ timestamp)
frames=[]
for p in sorted(HIST.glob("*_history.csv")):
    try:
        df = pd.read_csv(p)
        # expected slim columns; add if missing
        for c in ["provider","region","gpu_model","type","duration","gpu_count",
                  "price_hourly_usd","source_url","fetched_at_utc"]:
            if c not in df.columns: df[c] = None
        frames.append(df)
    except Exception as e:
        print(f"[WARN] {p.name}: {e}")

if not frames:
    raise SystemExit("No history files in docs/data/history/*.csv — ensure scrapers append to history.")

raw = pd.concat(frames, ignore_index=True)

# Normalize
raw["gpu_model"] = raw["gpu_model"].astype(str).str.upper().str.strip()
raw["region"]    = raw["region"].astype(str).replace({"nan":None}).fillna("Global").str.strip()
raw["type"]      = raw["type"].astype(str).replace({"nan":None}).fillna("On-Demand").str.strip()
raw["price_hourly_usd"] = pd.to_numeric(raw["price_hourly_usd"], errors="coerce")
raw["fetched_at_utc"]   = pd.to_datetime(raw["fetched_at_utc"], errors="coerce", utc=True)
raw = raw.dropna(subset=["provider","gpu_model","price_hourly_usd","fetched_at_utc"])

# Filter to H100/H200 & sane band (let hyperscalers exceed band)
raw = raw[(raw["gpu_model"].isin(["H100","H200"])) &
          ((raw["price_hourly_usd"].between(0.5, 10.0)) | (raw["provider"].isin(["AWS","Azure","GCP"])))]

# Daily snapshot (latest per provider/model/region/type each UTC date)
raw["date"] = raw["fetched_at_utc"].dt.floor("D")
keys = ["date","provider","region","gpu_model","type","duration"]
daily = (raw.sort_values("fetched_at_utc")
            .drop_duplicates(subset=keys, keep="last")
            .reset_index(drop=True))

# Recompute DAILY market baselines (On-Demand only)
od = daily[daily["type"].eq("On-Demand")].copy()
grp = ["date","gpu_model","region"]
g = od.groupby(grp)["price_hourly_usd"]
market = pd.DataFrame({
    "market_count": g.size(),
    "market_median": g.median(),
    "market_mean": g.mean(),
    "market_p25": g.quantile(0.25),
    "market_p75": g.quantile(0.75),
}).reset_index()
market["market_iqr"] = (market["market_p75"] - market["market_p25"]).round(6)

# Join baselines back to all rows on same (date, model, region)
panel = daily.merge(market, on=["date","gpu_model","region"], how="left")

# Compute daily Price-IQ (price-only)
panel["price_iq"] = (100 * (panel["market_median"] / panel["price_hourly_usd"])) \
                        .clip(50,150).round(1)

# Keep reasonable coverage only (need ≥2 providers to form a market that day)
panel = panel[panel["market_count"] >= 2].reset_index(drop=True)

# Save panel for further analysis
panel_out = DER / "backtest_panel_daily.csv"
panel.to_csv(panel_out, index=False)
print("Saved daily panel ->", panel_out, "rows:", len(panel))
display(panel.head())
# =================================================================================================


Saved daily panel -> docs/data/derived/backtest_panel_daily.csv rows: 18


,provider,region,gpu_model,type,duration,gpu_count,price_hourly_usd,source_url,fetched_at_utc,date,market_count,market_median,market_mean,market_p25,market_p75,market_iqr,price_iq
0,Nebius,Global,H100,On-Demand,1h,NaN,2.00,https://nebius.com/prices,2025-09-04 11:42:58.443649+00:00,2025-09-04 00:00:00+00:00,8,2.245,3.085938,1.8125,2.7000,0.8875,112.2
1,Nebius,Global,H200,On-Demand,1h,NaN,2.30,https://nebius.com/prices,2025-09-04 11:42:58.443649+00:00,2025-09-04 00:00:00+00:00,4,2.475,2.885000,2.4125,2.9475,0.5350,107.6
2,Vast.ai,Global,H100,On-Demand,1h,NaN,1.25,https://vast.ai/products/gpu-cloud,2025-09-04 12:42:14+00:00,2025-09-04 00:00:00+00:00,8,2.245,3.085938,1.8125,2.7000,0.8875,150.0
3,Shadeform,Global,H200,On-Demand,1h,NaN,2.45,https://www.shadeform.ai/,2025-09-04 12:45:03+00:00,2025-09-04 00:00:00+00:00,4,2.475,2.885000,2.4125,2.9475,0.5350,101.0
4,Shadeform,Global,H100,On-Demand,1h,NaN,10.00,https://www.shadeform.ai/,2025-09-04 12:45:03+00:00,2025-09-04 00:00:00+00:00,8,2.245,3.085938,1.8125,2.7000,0.8875,50.0


In [21]:
# =========================== BACKTEST METRICS & CALIBRATION ===========================
from pathlib import Path
import pandas as pd, numpy as np, json

DER = Path("docs/data/derived")
panel = pd.read_csv(DER/"backtest_panel_daily.csv", parse_dates=["date"])

# For each (date, model, region), compare ranking by Price-IQ vs true price
def per_bucket(df):
    # ground truth: cheapest by price that day
    truth = df.sort_values("price_hourly_usd").reset_index(drop=True)
    by_iq = df.sort_values("price_iq", ascending=False).reset_index(drop=True)
    top_price_provider = truth.loc[0,"provider"]
    top_iq_provider    = by_iq.loc[0,"provider"]
    hit = (top_price_provider == top_iq_provider)

    # regret for a workload (gpus*hours*util) cancels out; use per-GPU-hour regret
    regret = by_iq.loc[0,"price_hourly_usd"] - truth.loc[0,"price_hourly_usd"]   # ≥ 0 when IQ picks worse
    return pd.Series({
        "providers": len(df),
        "top1_hit": float(hit),
        "regret_per_gpu_hour": float(max(0.0, regret)),
        "spearman_corr": df[["price_iq","price_hourly_usd"]].rank().corr(method="spearman").iloc[0,1]
    })

grp = ["date","gpu_model","region"]
metrics_daily = panel.groupby(grp, group_keys=False).apply(per_bucket).reset_index()

# Aggregate headline metrics per model
summary = (metrics_daily.groupby(["gpu_model"])
           .agg(top1_acc=("top1_hit","mean"),
                avg_regret_per_gpu_hr=("regret_per_gpu_hour","mean"),
                median_spearman=("spearman_corr","median"),
                days=("top1_hit","size"))
           .reset_index())

# Calibration table: how Price-IQ maps to savings vs median (should be monotonic)
panel["savings_vs_median_pct"] = (panel["market_median"]/panel["price_hourly_usd"])
panel["iq_bucket"] = (panel["price_iq"]//5*5).astype(int)  # bucket by 5-pt steps
calibration = (panel.groupby(["gpu_model","iq_bucket"])
               .agg(mean_savings_ratio=("savings_vs_median_pct","mean"),
                    count=("savings_vs_median_pct","size"))
               .reset_index())
calibration["mean_savings_pct"] = (calibration["mean_savings_ratio"]-1.0)*100
calibration = calibration.sort_values(["gpu_model","iq_bucket"])

# Save artifacts
metrics_daily_out = DER/"backtest_metrics_daily.csv"
summary_out       = DER/"backtest_summary.csv"
calib_out         = DER/"backtest_calibration.csv"
metrics_daily.to_csv(metrics_daily_out, index=False)
summary.to_csv(summary_out, index=False)
calibration.to_csv(calib_out, index=False)

# Small JSON for CI/dashboard
report = {
    "models": {
        m: {
            "top1_accuracy": round(float(summary.set_index("gpu_model").loc[m,"top1_acc"]), 3)
                               if m in summary["gpu_model"].values else None,
            "avg_regret_per_gpu_hour": round(float(summary.set_index("gpu_model").loc[m,"avg_regret_per_gpu_hr"]), 4)
                               if m in summary["gpu_model"].values else None,
            "median_spearman": round(float(summary.set_index("gpu_model").loc[m,"median_spearman"]), 3)
                               if m in summary["gpu_model"].values else None,
            "days_evaluated": int(summary.set_index("gpu_model").loc[m,"days"])
                               if m in summary["gpu_model"].values else 0
        } for m in sorted(panel["gpu_model"].unique())
    }
}
(DER/"backtest_report.json").write_text(json.dumps(report, indent=2))

print("Saved:",
      "\n -", metrics_daily_out,
      "\n -", summary_out,
      "\n -", calib_out,
      "\n -", DER/"backtest_report.json")
print("\nSummary:")
display(summary)
print("\nCalibration preview:")
display(calibration.head(20))
# ======================================================================================


Saved: 
 - docs/data/derived/backtest_metrics_daily.csv 
 - docs/data/derived/backtest_summary.csv 
 - docs/data/derived/backtest_calibration.csv 
 - docs/data/derived/backtest_report.json

Summary:


,gpu_model,top1_acc,avg_regret_per_gpu_hr,median_spearman,days
0,H100,0.0,0.5025,-0.997725,1
1,H200,1.0,0.0000,-1.000000,1



Calibration preview:


,gpu_model,iq_bucket,mean_savings_ratio,count,mean_savings_pct
0,H100,50,0.224500,1,-77.550000
1,H100,55,0.575641,1,-42.435897
2,H100,70,0.719551,1,-28.044872
3,H100,75,0.766212,1,-23.378840
4,H100,85,0.883858,1,-11.614173
5,H100,95,0.986932,2,-1.306763
6,H100,100,1.002232,1,0.223214
7,H100,110,1.122500,1,12.250000
8,H100,150,2.399672,2,139.967224
9,H200,55,0.576923,1,-42.307692


In [22]:
# ================= BACKTEST (robust QA & coverage) =================
from pathlib import Path
import pandas as pd, numpy as np

HIST = Path("docs/data/history")
DER  = Path("docs/data/derived"); DER.mkdir(parents=True, exist_ok=True)

frames=[]
for p in sorted(HIST.glob("*_history.csv")):
    try:
        df = pd.read_csv(p)
        for c in ["provider","region","gpu_model","type","duration","gpu_count",
                  "price_hourly_usd","source_url","fetched_at_utc"]:
            if c not in df.columns: df[c] = None
        frames.append(df)
    except Exception as e:
        print("[WARN]", p.name, e)

if not frames:
    raise SystemExit("No history files in docs/data/history/*.csv — run scrapers with history first.")

raw = pd.concat(frames, ignore_index=True)

# Normalize
raw["gpu_model"] = raw["gpu_model"].astype(str).str.upper().str.strip()
raw["region"]    = raw["region"].astype(str).replace({"nan":None}).fillna("Global").str.strip()
raw["type"]      = raw["type"].astype(str).replace({"nan":None}).fillna("On-Demand").str.strip()
raw["price_hourly_usd"] = pd.to_numeric(raw["price_hourly_usd"], errors="coerce")
raw["fetched_at_utc"]   = pd.to_datetime(raw["fetched_at_utc"], errors="coerce", utc=True)
raw = raw.dropna(subset=["provider","gpu_model","price_hourly_usd","fetched_at_utc"])

# --- QA (match production aggregator) ---
raw = raw[raw["gpu_model"].isin(["H100","H200"])]

# sane price band for market players; allow hyperscalers outside band
band = raw["price_hourly_usd"].between(0.5, 10.0)
hypers = raw["provider"].isin(["AWS","Azure","GCP"])
raw = raw[band | hypers]

# Shadeform: drop marketing blurbs (e.g., ~10 $/GPU-hr H100)
raw = raw[~((raw["provider"].eq("Shadeform")) &
            (raw["gpu_model"].eq("H100")) &
            (raw["price_hourly_usd"] > 6.0))]

# Only On-Demand for baseline
raw = raw[raw["type"].eq("On-Demand")].copy()

# Daily snapshot (latest per provider/model/region/date)
raw["date"] = raw["fetched_at_utc"].dt.floor("D")
keys = ["date","provider","region","gpu_model","type","duration"]
daily = (raw.sort_values("fetched_at_utc")
            .drop_duplicates(subset=keys, keep="last")
            .reset_index(drop=True))

# Compute robust daily baselines
grp = ["date","gpu_model","region"]
g = daily.groupby(grp)["price_hourly_usd"]

def trimmed_median(x, trim=0.1):
    # drop top/bottom 10% when sample is big enough
    if len(x) >= 6:
        lo, hi = x.quantile([trim, 1-trim])
        x = x[(x>=lo)&(x<=hi)]
    return x.median()

market = pd.DataFrame({
    "market_count": g.size(),
    "market_median": g.apply(trimmed_median),   # more robust
    "market_mean": g.mean(),
    "market_p25": g.quantile(0.25),
    "market_p75": g.quantile(0.75),
}).reset_index()
market["market_iqr"] = (market["market_p75"] - market["market_p25"]).round(6)

panel = daily.merge(market, on=grp, how="left")

# Require real coverage (≥ 3 providers in the bucket)
panel = panel[panel["market_count"] >= 3].reset_index(drop=True)

# Price-IQ (price-only)
panel["price_iq"] = (100 * (panel["market_median"] / panel["price_hourly_usd"])) \
                        .clip(50,150).round(1)

panel_out = DER / "backtest_panel_daily.csv"
panel.to_csv(panel_out, index=False)
print("Saved robust daily panel ->", panel_out, "rows:", len(panel))
display(panel.head(10))
# ================================================================



Saved robust daily panel -> docs/data/derived/backtest_panel_daily.csv rows: 11


,provider,region,gpu_model,type,duration,gpu_count,price_hourly_usd,source_url,fetched_at_utc,date,market_count,market_median,market_mean,market_p25,market_p75,market_iqr,price_iq
0,Nebius,Global,H100,On-Demand,1h,NaN,2.00,https://nebius.com/prices,2025-09-04 11:42:58.443649+00:00,2025-09-04 00:00:00+00:00,7,2.240,2.205186,1.74815,2.2750,0.52685,112.0
1,Nebius,Global,H200,On-Demand,1h,NaN,2.30,https://nebius.com/prices,2025-09-04 11:42:58.443649+00:00,2025-09-04 00:00:00+00:00,4,2.475,2.885000,2.41250,2.9475,0.53500,107.6
2,Vast.ai,Global,H100,On-Demand,1h,NaN,1.25,https://vast.ai/products/gpu-cloud,2025-09-04 12:42:14+00:00,2025-09-04 00:00:00+00:00,7,2.240,2.205186,1.74815,2.2750,0.52685,150.0
3,Shadeform,Global,H200,On-Demand,1h,NaN,2.45,https://www.shadeform.ai/,2025-09-04 12:45:03+00:00,2025-09-04 00:00:00+00:00,4,2.475,2.885000,2.41250,2.9475,0.53500,101.0
4,Paperspace,Global,H100,On-Demand,1h,NaN,2.24,https://www.paperspace.com/pricing,2025-09-04 13:05:49+00:00,2025-09-04 00:00:00+00:00,7,2.240,2.205186,1.74815,2.2750,0.52685,100.0
5,TensorDock,Global,H100,On-Demand,1h,1.0,2.25,https://tensordock.com/gpu-h100,2025-09-04 13:16:31+00:00,2025-09-04 00:00:00+00:00,7,2.240,2.205186,1.74815,2.2750,0.52685,99.6
6,Hydra Host (Brokkr),Global,H200,On-Demand,1h,1.0,2.50,https://brokkr.hydrahost.com/inventory,2025-09-04 13:22:03+00:00,2025-09-04 00:00:00+00:00,4,2.475,2.885000,2.41250,2.9475,0.53500,99.0
7,Hydra Host (Brokkr),Global,H100,On-Demand,1h,1.0,2.30,https://brokkr.hydrahost.com/inventory,2025-09-04 13:22:03+00:00,2025-09-04 00:00:00+00:00,7,2.240,2.205186,1.74815,2.2750,0.52685,97.4
8,CrusoeCloud,Global,H100,On-Demand,1h,1.0,3.90,https://www.crusoe.ai/cloud/pricing,2025-09-04 13:27:24+00:00,2025-09-04 00:00:00+00:00,7,2.240,2.205186,1.74815,2.2750,0.52685,57.4
9,CrusoeCloud,Global,H200,On-Demand,1h,1.0,4.29,https://www.crusoe.ai/cloud/pricing,2025-09-04 13:27:24+00:00,2025-09-04 00:00:00+00:00,4,2.475,2.885000,2.41250,2.9475,0.53500,57.7


In [23]:
from pathlib import Path
import pandas as pd, numpy as np, json

DER = Path("docs/data/derived")
panel = pd.read_csv(DER/"backtest_panel_daily.csv", parse_dates=["date"])

if panel.empty:
    raise SystemExit("Backtest panel empty after QA/coverage. Reduce coverage gate to >=2 or collect more days.")

def per_bucket(df):
    truth = df.sort_values("price_hourly_usd").reset_index(drop=True)
    pred  = df.sort_values("price_iq", ascending=False).reset_index(drop=True)
    return pd.Series({
        "providers": len(df),
        "top1_hit": float(truth.loc[0,"provider"] == pred.loc[0,"provider"]),
        "regret_per_gpu_hour": float(max(0.0, pred.loc[0,"price_hourly_usd"] - truth.loc[0,"price_hourly_usd"])),
        "spearman_corr": df[["price_iq","price_hourly_usd"]].rank().corr(method="spearman").iloc[0,1]
    })

grp = ["date","gpu_model","region"]
metrics_daily = panel.groupby(grp, group_keys=False).apply(per_bucket).reset_index()

summary = (metrics_daily.groupby("gpu_model")
           .agg(top1_acc=("top1_hit","mean"),
                avg_regret_per_gpu_hr=("regret_per_gpu_hour","mean"),
                median_spearman=("spearman_corr","median"),
                days=("top1_hit","size"))
           .reset_index())

metrics_daily.to_csv(DER/"backtest_metrics_daily.csv", index=False)
summary.to_csv(DER/"backtest_summary.csv", index=False)
(DER/"backtest_report.json").write_text(json.dumps(summary.to_dict(orient="records"), indent=2))

print("Saved: backtest_metrics_daily.csv, backtest_summary.csv, backtest_report.json")
display(summary)


Saved: backtest_metrics_daily.csv, backtest_summary.csv, backtest_report.json


,gpu_model,top1_acc,avg_regret_per_gpu_hr,median_spearman,days
0,H100,1.0,0.0,-1.0,1
1,H200,1.0,0.0,-1.0,1


In [24]:
# ===================== PRICE-IQ / ROI PREFLIGHT & BUNDLE ======================
from pathlib import Path
import pandas as pd, numpy as np, json, sys
from datetime import datetime, timezone

DER = Path("docs/data/derived"); LATEST = Path("docs/data/latest")
DER.mkdir(parents=True, exist_ok=True)

required_latest = [
    # not hard-failing on missing individual providers—just reporting
    "brokkr_latest.csv","coreweave_latest.csv","crusoecloud_latest.csv",
    "lambda_labs_latest.csv","nebius_latest.csv","paperspace_latest.csv",
    "runpod_latest.csv","tensordock_latest.csv","vastai_latest.csv",
    "voltagepark_latest.csv"
]
required_derived = [
    "provider_scores_latest.csv",
    "market_index.csv",
    "price_iq_explainable.csv",   # CSV version of leaderboard
    "price_iq_latest.json",       # JSON leaderboard for site
    "roi_comparison_latest.csv",
    "roi_comparison_latest.json",
    "backtest_panel_daily.csv",
    "backtest_summary.csv"
]

def ok_file(p: Path, min_bytes=200):
    return p.exists() and p.stat().st_size >= min_bytes

# 1) Files present?
missing_latest  = [f for f in required_latest  if not (LATEST/f).exists()]
missing_derived = [f for f in required_derived if not ok_file(DER/f)]

# 2) Schema checks
problems = []
def need_cols(df, cols, name):
    miss = [c for c in cols if c not in df.columns]
    if miss: problems.append(f"{name}: missing cols {miss}")

scores_p = DER/"provider_scores_latest.csv"
market_p = DER/"market_index.csv"
iq_p     = DER/"price_iq_explainable.csv"
roi_p    = DER/"roi_comparison_latest.csv"

try:
    scores = pd.read_csv(scores_p)
    need_cols(scores,
              ["provider","region","gpu_model","type","price_hourly_usd",
               "market_median","price_score","source_url"],
              "provider_scores_latest.csv")
except Exception as e:
    problems.append(f"Read fail: {scores_p.name}: {e}")
    scores = pd.DataFrame()

try:
    roi = pd.read_csv(roi_p)
    need_cols(roi,
              ["gpu_model","provider","region","type","price_hourly_usd",
               "total_cost","savings_vs_median","source_url"],
              "roi_comparison_latest.csv")
except Exception as e:
    problems.append(f"Read fail: {roi_p.name}: {e}")
    roi = pd.DataFrame()

# 3) Coverage & bands
coverage = {}
if not scores.empty:
    od = scores.query("type == 'On-Demand'").copy()
    od["gpu_model"] = od["gpu_model"].astype(str).str.upper()
    cov = (od.query("gpu_model in ['H100','H200']")
             .groupby("gpu_model")["provider"].nunique().to_dict())
    coverage = {"H100": cov.get("H100",0), "H200": cov.get("H200",0)}
    # sane per-GPU bands (hyperscalers exempt)
    band = od["price_hourly_usd"].between(0.5, 10.0)
    hyper = od["provider"].isin(["AWS","Azure","GCP"])
    band_ok_rate = float(((band | hyper).sum()) / max(1, len(od)))
else:
    band_ok_rate = 0.0

# 4) IQ calibration quick check (IQ should inversely correlate with price)
iq_ok = None
try:
    iq = pd.read_csv(iq_p)
    if not iq.empty:
        r = iq[["price_iq","price_hourly_usd"]].rank().corr(method="spearman").iloc[0,1]
        iq_ok = (r <= -0.6)  # strongly negative = good
except Exception as e:
    problems.append(f"Read fail: {iq_p.name}: {e}")

# 5) Build a compact “dashboard bundle” (optional, but handy for a static site)
def to_records(df, cols):
    return json.loads(df[cols].to_json(orient="records"))

bundle = {}
if ok_file(DER/"price_iq_explainable.csv"):
    iqv = pd.read_csv(DER/"price_iq_explainable.csv")
    iqv["gpu_model"] = iqv["gpu_model"].astype(str).str.upper()
    bundle["leaderboard"] = {
        m: to_records(
            iqv[iqv["gpu_model"]==m]
               .sort_values(["region","price_iq"], ascending=[True,False])
               .reset_index(drop=True),
            ["gpu_model","provider","region","type","price_hourly_usd","market_median","price_iq","source_url"]
        )
        for m in sorted(iqv["gpu_model"].dropna().unique())
    }
if ok_file(DER/"roi_comparison_latest.csv"):
    rod = roi.copy()
    if not rod.empty:
        bundle["roi"] = to_records(
            rod.sort_values(["gpu_model","total_cost","provider"]),
            ["gpu_model","provider","region","type","price_hourly_usd","total_cost","savings_vs_median","source_url"]
        )

manifest = {
    "generated_at_utc": datetime.now(timezone.utc).isoformat(timespec="seconds"),
    "missing_latest": missing_latest,
    "missing_derived": missing_derived,
    "schema_problems": problems,
    "coverage_on_demand_unique_providers": coverage,
    "band_ok_rate": round(band_ok_rate, 3),
    "iq_inverse_corr_pass": bool(iq_ok) if iq_ok is not None else None,
    "counts": {
        "scores_rows": int(len(scores)) if not scores.empty else 0,
        "roi_rows": int(len(roi)) if not roi.empty else 0
    }
}
(DER/"manifest.json").write_text(json.dumps(manifest, indent=2))
(DER/"dashboard_bundle.json").write_text(json.dumps(bundle, indent=2))

print("=== Preflight summary ===")
print(json.dumps(manifest, indent=2))
print("\nLeaderboard preview:")
try:
    display(pd.read_csv(DER/"price_iq_explainable.csv")
              .sort_values(["gpu_model","price_iq"], ascending=[True,False])
              [["gpu_model","provider","region","type","price_hourly_usd","market_median","price_iq"]]
              .head(10))
except Exception:
    pass
print("\nROI preview:")
if not roi.empty:
    display(roi.sort_values(["gpu_model","total_cost"]).head(10))
# =============================================================================


=== Preflight summary ===
{
  "generated_at_utc": "2025-09-08T13:14:09+00:00",
  "missing_latest": [
    "lambda_labs_latest.csv",
    "runpod_latest.csv"
  ],
  "missing_derived": [
    "backtest_summary.csv"
  ],
  "schema_problems": [],
  "coverage_on_demand_unique_providers": {
    "H100": 8,
    "H200": 4
  },
  "band_ok_rate": 0.923,
  "iq_inverse_corr_pass": true,
  "counts": {
    "scores_rows": 19,
    "roi_rows": 8
  }
}

Leaderboard preview:


,gpu_model,provider,region,type,price_hourly_usd,market_median,price_iq
3,H100,VoltagePark,US,On-Demand,1.99,10.995,150.0
4,H100,Vast.ai,Global,On-Demand,1.25,2.245,150.0
1,H100,Nebius,Global,On-Demand,2.00,2.245,111.1
6,H100,Paperspace,Global,On-Demand,2.24,2.245,100.2
8,H100,TensorDock,Global,On-Demand,2.25,2.245,99.8
9,H100,Hydra Host (Brokkr),Global,On-Demand,2.30,2.245,97.8
14,H100,CrusoeCloud,Global,Reserved-3y,2.54,2.245,89.4
15,H100,CrusoeCloud,Global,Reserved-1y,2.93,2.245,78.7
13,H100,CrusoeCloud,Global,Reserved-6m,3.12,2.245,74.5
16,H100,CrusoeCloud,Global,On-Demand,3.90,2.245,61.4



ROI preview:


,gpu_model,provider,region,type,price_hourly_usd,market_median,gpu_hours_eff,infra_cost,extra_fees,egress_cost,storage_cost,total_cost,delta_vs_cheapest,savings_vs_median,expected_revenue,gross_margin,roi_pct,source_url
0,H100,Vast.ai,Global,On-Demand,1.25,2.245,7200.0,9000.0,0.0,0.0,0.0,9000.0,0.0,7164.0,NaN,NaN,NaN,https://vast.ai/products/gpu-cloud
1,H100,VoltagePark,US,On-Demand,1.99,10.995,7200.0,14328.0,0.0,0.0,0.0,14328.0,5328.0,64836.0,NaN,NaN,NaN,https://dashboard.voltagepark.com/order/config...
2,H100,Nebius,Global,On-Demand,2.00,2.245,7200.0,14400.0,0.0,0.0,0.0,14400.0,5400.0,1764.0,NaN,NaN,NaN,https://nebius.com/prices
3,H100,Paperspace,Global,On-Demand,2.24,2.245,7200.0,16128.0,0.0,0.0,0.0,16128.0,7128.0,36.0,NaN,NaN,NaN,https://www.paperspace.com/pricing
4,H100,TensorDock,Global,On-Demand,2.25,2.245,7200.0,16200.0,0.0,0.0,0.0,16200.0,7200.0,-36.0,NaN,NaN,NaN,https://tensordock.com/gpu-h100
5,H100,Hydra Host (Brokkr),Global,On-Demand,2.30,2.245,7200.0,16560.0,0.0,0.0,0.0,16560.0,7560.0,-396.0,NaN,NaN,NaN,https://brokkr.hydrahost.com/inventory
6,H100,CrusoeCloud,Global,On-Demand,3.90,2.245,7200.0,28080.0,0.0,0.0,0.0,28080.0,19080.0,-11916.0,NaN,NaN,NaN,https://www.crusoe.ai/cloud/pricing
7,H100,CoreWeave,US,On-Demand,20.00,10.995,7200.0,144000.0,0.0,0.0,0.0,144000.0,135000.0,-64836.0,NaN,NaN,NaN,https://www.coreweave.com/pricing


In [32]:
train['price_type'].value_counts(), sorted(train['region'].unique()), sorted(train['cpu_platform'].unique())


(OnDemand    6
 Reserved    2
 Name: price_type, dtype: int64,
 ['Global'],
 ['Unknown'])

In [33]:
# ===============================================
# PRICEIQ REVERSE-ENGINEERING — SINGLE, CLEAN CELL
# ===============================================
# It:
#  1) Loads quotes CSVs (auto-discovers under docs/ and data/)
#  2) Harmonises columns to PriceIQ-like inputs
#  3) Trains OLS + Quantile models (p10,p25,p50,p75,p90)
#  4) Builds a prediction grid ONLY from categories observed in training
#  5) Predicts using training design_info (no column mismatches)
#  6) Writes artefacts to docs/data/derived/priceiq_model/
#
# Dependencies: pandas numpy statsmodels patsy scikit-learn matplotlib
# pip install pandas numpy statsmodels patsy scikit-learn matplotlib

import os, re, json, math, warnings
from pathlib import Path
from typing import List, Dict, Tuple

import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.regression.quantile_regression import QuantReg
from patsy import dmatrices, dmatrix, build_design_matrices
from sklearn.metrics import mean_absolute_error, r2_score
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")
np.random.seed(42)

# -------------------------
# CONFIG
# -------------------------
INPUT_FILES = [
    # If you know the exact CSVs, put them here; otherwise autodiscovery kicks in.
    "docs/data/derived/provider_quotes_latest.csv",
    "docs/data/derived/provider_quotes_history.csv",
    "docs/data/sfcompute_history.csv",
    "docs/sfcompute_grid/grid_history.csv",
    "sfcompute_grid/grid_history.csv",
    "data/sfcompute_history.csv",
    "docs/data/derived/sfcompute_history.csv"
]
OUT_DIR = Path("docs/data/derived/priceiq_model"); OUT_DIR.mkdir(parents=True, exist_ok=True)
QUANTILES = [0.10, 0.25, 0.50, 0.75, 0.90]
INCLUDE_PROVIDER_FE = True
INCLUDE_DURATION_TERM = True

# -------------------------
# HELPERS
# -------------------------
DURATION_MAP = {
    "1 hour": 1/24, "6 hours": 6/24, "12 hours": 0.5, "1 day": 1, "3 days": 3,
    "1 week": 7, "2 weeks": 14, "3 weeks": 21, "1 month": 30, "2 months": 60,
    "3 months": 90, "6 months": 180, "12 months": 365
}
CPU_PLAT_MAP = {"intel": "Intel", "amd": "AMD", "arm": "ARM"}
PRICE_TYPE_ALIASES = {"on-demand": "OnDemand", "ondemand": "OnDemand", "on demand": "OnDemand",
                      "reserved": "Reserved", "commit": "Reserved", "committed": "Reserved",
                      "spot": "Spot", "market": "Spot"}
REGION_ALIASES = {"eu": "EU", "europe": "EU", "us": "US", "usa": "US", "na": "US", "uk": "UK", "gb": "UK"}
GPU_MODEL_ALIASES = {"nvidia h100": "H100", "h100": "H100", "nvidia h200": "H200", "h200": "H200",
                     "nvidia a100": "A100", "a100": "A100"}
GB_RE = re.compile(r"(\d+\.?\d*)\s*gb", re.I)

def first_path_that_exists(paths: List[str]) -> List[Path]:
    return [Path(p) for p in paths if Path(p).exists()]

def norm_price_type(x: str) -> str:
    if not isinstance(x, str): return None
    return PRICE_TYPE_ALIASES.get(x.lower().strip(), x.title())

def norm_region(x: str) -> str:
    if not isinstance(x, str): return None
    return REGION_ALIASES.get(x.lower().strip(), x.upper())

def norm_gpu_model(x: str) -> str:
    if not isinstance(x, str): return None
    return GPU_MODEL_ALIASES.get(x.lower().strip(), x.upper())

def extract_gb(s: str) -> float:
    if not isinstance(s, str): return np.nan
    m = GB_RE.search(s); return float(m.group(1)) if m else np.nan

def to_float_safe(x):
    try: return float(x)
    except Exception: return np.nan

def duration_to_days(x) -> float:
    if x is None or (isinstance(x, float) and math.isnan(x)): return np.nan
    if isinstance(x, (int, float)): return float(x)
    s = str(x).strip().lower()
    if s in DURATION_MAP: return float(DURATION_MAP[s])
    m = re.match(r"(\d+\.?\d*)([dwm])", s)
    if m:
        val, unit = float(m.group(1)), m.group(2)
        return val * (1 if unit=='d' else 7 if unit=='w' else 30)
    try: return float(s)
    except Exception: return np.nan

# -------------------------
# LOAD & HARMONISE
# -------------------------
CANDIDATE_COL_MAP = {
    "provider": ["provider","vendor","cloud"],
    "region": ["region","geo","location"],
    "gpu_model": ["gpu_model","gpu_type","gpu"],
    "price_type": ["price_type","type","market_type"],
    "price_per_gpu_hr": ["usd_per_gpu_hr","price_hourly_usd","price_per_gpu_hr","price_usd_per_hr"],
    "gpu_ram_gb": ["gpu_ram_gb","vram_gb","gpu_mem_gb","gpu_ram"],
    "cpu_platform": ["cpu_platform","cpu_vendor","platform"],
    "cpu_cores_eff": ["cpu_cores_eff","cpu_cores","vcpu"],
    "cpu_ram_gb": ["cpu_ram_gb","system_ram_gb","ram_gb"],
    "timestamp": ["timestamp","ts_utc","time","date"],
    "reserved_duration": ["reserved_duration","duration","term"]
}

def harmonise_columns(df: pd.DataFrame) -> pd.DataFrame:
    cols = {c.lower(): c for c in df.columns}
    out = {}
    for target, candidates in CANDIDATE_COL_MAP.items():
        src = None
        for cand in candidates:
            if cand.lower() in cols:
                src = cols[cand.lower()]; break
        out[target] = src
    uni = pd.DataFrame()
    for k, src in out.items():
        uni[k] = df[src] if src is not None else np.nan

    # Normalise
    uni["provider"] = uni["provider"].astype(str).str.strip().replace({"nan": np.nan}).fillna("Unknown")
    uni["region"] = uni["region"].map(norm_region).fillna("Global")
    uni["gpu_model"] = uni["gpu_model"].map(norm_gpu_model)
    uni["price_type"] = uni["price_type"].map(norm_price_type)
    uni["price_per_gpu_hr"] = uni["price_per_gpu_hr"].apply(to_float_safe)
    uni["timestamp"] = pd.to_datetime(uni["timestamp"], errors="coerce", utc=True)

    # RAM fallbacks
    if uni["gpu_ram_gb"].isna().all() and "gpu_model" in uni:
        uni["gpu_ram_gb"] = uni["gpu_model"].apply(extract_gb)

    uni["cpu_platform"] = uni["cpu_platform"].astype(str).str.lower().map(CPU_PLAT_MAP).fillna("Unknown")
    for c in ["gpu_ram_gb","cpu_cores_eff","cpu_ram_gb"]:
        uni[c] = pd.to_numeric(uni[c], errors="coerce")

    # Duration
    uni["duration_days"] = np.nan
    if out.get("reserved_duration") is not None:
        uni["duration_days"] = uni["reserved_duration"].apply(duration_to_days)

    # Infer price_type from duration if missing
    if uni["price_type"].isna().any():
        uni.loc[uni["price_type"].isna(), "price_type"] = np.where(uni["duration_days"] >= 7, "Reserved", "OnDemand")

    # Keep meaningful rows
    uni = uni.dropna(subset=["gpu_model","price_type","region","price_per_gpu_hr"], how="any")
    return uni

def load_quotes(paths: List[str]) -> pd.DataFrame:
    files = first_path_that_exists(paths)
    if not files:
        # autodiscover
        search_roots = [Path("docs"), Path("data"), Path("."), Path("docs/data"), Path("docs/data/derived")]
        cand_files = []
        for root in search_roots:
            if root.exists():
                for p in root.rglob("*.csv"):
                    name = p.name.lower()
                    if any(k in name for k in ["sfcompute","quotes","history","grid"]):
                        cand_files.append(p)
        def has_price_cols(p: Path) -> bool:
            try:
                head = pd.read_csv(p, nrows=5)
                cols = [c.lower() for c in head.columns]
                return any(k in cols for k in ["usd_per_gpu_hr","price_hourly_usd","price_per_gpu_hr"]) and \
                       any(x in cols for x in ["gpu_model","gpu_type","gpu"])
            except Exception:
                return False
        files = [p for p in cand_files if has_price_cols(p)]
        if not files:
            raise FileNotFoundError("No input CSVs found. Put a quotes/history CSV under docs/ or data/ (e.g., docs/data/sfcompute_history.csv)")
    frames = []
    for f in files:
        try:
            raw = pd.read_csv(f)
            uni = harmonise_columns(raw)
            if not uni.empty:
                frames.append(uni)
                print(f"[load_quotes] Loaded {f} -> {len(uni)} rows")
        except Exception as e:
            print(f"[WARN] Could not load {f}: {e}")
    if not frames:
        raise RuntimeError("No valid frames after harmonisation.")
    df = pd.concat(frames, ignore_index=True).drop_duplicates()
    return df

# -------------------------
# MODEL PREP
# -------------------------
def prepare_for_model(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    for c in ["gpu_model","price_type","region","cpu_platform","provider"]:
        if c not in df.columns: df[c] = "Unknown"
        df[c] = df[c].astype(str).fillna("Unknown")
    num_cols = ["cpu_cores_eff","cpu_ram_gb","gpu_ram_gb","duration_days"]
    for c in num_cols:
        if c not in df.columns: df[c] = np.nan
        df[c] = pd.to_numeric(df[c], errors="coerce")
    if df["duration_days"].isna().all(): df["duration_days"] = 0.0
    med_by_gpu = df.groupby("gpu_model")[num_cols].transform(lambda s: s.fillna(s.median()))
    for c in num_cols:
        df[c] = med_by_gpu[c].fillna(df[c].median())
    df = df.dropna(subset=["price_per_gpu_hr"]) 
    df = df[(df["price_per_gpu_hr"] > 0) & (df["price_per_gpu_hr"] < 100.0)]
    return df

def build_formula(include_provider_fe=True, include_duration_term=True) -> str:
    base = "price_per_gpu_hr ~ C(gpu_model) + C(price_type) + C(region) + C(cpu_platform) + cpu_cores_eff + cpu_ram_gb + gpu_ram_gb"
    if include_provider_fe:
        base += " + C(provider)"
    if include_duration_term:
        base += " + np.log1p(duration_days)"
    return base

# -------------------------
# FIT
# -------------------------
def fit_ols(df: pd.DataFrame, formula: str):
    y, X = dmatrices(formula, data=df, return_type='dataframe')
    model = sm.OLS(y, X).fit(cov_type='HC1')
    return model, X, y, X.design_info

def fit_quantiles(df: pd.DataFrame, formula_rhs: str, taus: List[float]) -> Tuple[Dict[float, sm.regression.linear_model.RegressionResultsWrapper], object]:
    y = df['price_per_gpu_hr']
    X = dmatrix(formula_rhs, data=df, return_type='dataframe')
    models = {}
    for tau in taus:
        try:
            qr = QuantReg(y, X).fit(q=tau)
            models[tau] = qr
        except Exception as e:
            print(f"[WARN] Quantile {tau} failed: {e}")
    return models, X.design_info

# -------------------------
# GRID + PREDICT (category-safe)
# -------------------------
def unique_nonnull(series: pd.Series, topk: int = 20) -> List[str]:
    vals = [v for v in series.dropna().astype(str).unique().tolist() if v]
    return vals[:topk]

def build_prediction_grid(df: pd.DataFrame) -> pd.DataFrame:
    # Use only categories SEEN in training to avoid Patsy unseen-level errors
    gpus = unique_nonnull(df['gpu_model'])
    types = unique_nonnull(df['price_type'])
    regions = unique_nonnull(df['region'])
    platforms = unique_nonnull(df['cpu_platform'])

    med = df.groupby('gpu_model')[['gpu_ram_gb','cpu_cores_eff','cpu_ram_gb']].median()
    rows = []
    for gpu in gpus:
        mrow = med.loc[gpu] if gpu in med.index else df[['gpu_ram_gb','cpu_cores_eff','cpu_ram_gb']].median()
        for pt in types:
            for reg in regions:
                for plat in platforms:
                    rows.append({
                        'gpu_model': gpu,
                        'price_type': pt,
                        'region': reg,
                        'cpu_platform': plat,
                        'gpu_ram_gb': float(mrow.get('gpu_ram_gb', np.nan)),
                        'cpu_cores_eff': float(mrow.get('cpu_cores_eff', np.nan)),
                        'cpu_ram_gb': float(mrow.get('cpu_ram_gb', np.nan)),
                        'duration_days': 0.0  # set non-zero if you want reserved curve
                    })
    return pd.DataFrame(rows)

def predict_with_models(df_design: pd.DataFrame, ols, quantile_models: Dict[float, sm.regression.linear_model.RegressionResultsWrapper], design_info_ols=None, design_info_q=None) -> pd.DataFrame:
    if df_design is None or df_design.empty:
        return pd.DataFrame()
    # Build matrices using training design schema
    Xg = build_design_matrices([design_info_ols], df_design, return_type='dataframe')[0] if design_info_ols is not None else None
    out = df_design.copy()
    if Xg is not None and Xg.shape[1] == len(ols.params):
        out['pred_ols'] = ols.predict(Xg)
    else:
        print("[predict] OLS design mismatch; filling NaN")
        out['pred_ols'] = np.nan
    if quantile_models:
        Xq = build_design_matrices([design_info_q], df_design, return_type='dataframe')[0] if design_info_q is not None else Xg
        for tau, m in quantile_models.items():
            col = f'pred_q{int(tau*100)}'
            try:
                if Xq is not None and Xq.shape[1] == len(m.params):
                    out[col] = m.predict(Xq)
                else:
                    out[col] = np.nan
            except Exception as e:
                print(f"[predict] Quantile {tau} failed: {e}")
                out[col] = np.nan
    return out

# -------------------------
# EXPORTS + PLOTS
# -------------------------
def evaluate_and_export(df: pd.DataFrame, ols, X, y, quantile_models):
    y_hat = ols.predict(X).values.ravel()
    r2 = r2_score(y, y_hat)
    mae = mean_absolute_error(y, y_hat)
    mape = float(np.mean(np.abs((y.values.ravel() - y_hat) / np.clip(y.values.ravel(), 1e-8, None))))

    coef = (pd.Series(ols.params, name='coef').to_frame().assign(se=ols.bse).reset_index().rename(columns={'index':'term'}))
    coef.to_csv(OUT_DIR / 'coefficients_ols.csv', index=False)

    qcoef = []
    for tau, m in quantile_models.items():
        s = pd.Series(m.params, name='coef').to_frame().reset_index().rename(columns={'index':'term'})
        s['tau'] = tau; qcoef.append(s)
    if qcoef:
        pd.concat(qcoef, ignore_index=True).to_csv(OUT_DIR / 'coefficients_quantiles.csv', index=False)

    with open(OUT_DIR / 'metrics.json', 'w') as f:
        json.dump({"r2": float(r2), "mae": float(mae), "mape": float(mape)}, f, indent=2)
    print(f"OLS R^2={r2:.3f}  MAE=${mae:.2f}/GPU-hr  MAPE={mape:.2%}")

def plot_price_bands(obs: pd.DataFrame, preds: pd.DataFrame, gpu: str, savepath: Path):
    sub_obs = obs[obs['gpu_model'] == gpu]
    sub_pred = preds[preds['gpu_model'] == gpu]
    if sub_obs.empty or sub_pred.empty:
        print(f"[plot] No data for {gpu}")
        return
    plt.figure(figsize=(10,6))
    g = (sub_obs.groupby(['region','price_type'])['price_per_gpu_hr'].median().reset_index())
    for _, r in g.iterrows():
        plt.scatter(f"{r['region']}\n{r['price_type']}", r['price_per_gpu_hr'], alpha=0.7)
    for _, r in sub_pred.iterrows():
        x = f"{r['region']}\n{r['price_type']}"
        p10, p50, p90 = r.get('pred_q10', np.nan), r.get('pred_q50', np.nan), r.get('pred_q90', np.nan)
        if not (np.isnan(p10) or np.isnan(p90)):
            plt.vlines(x, p10, p90)
        if not np.isnan(p50):
            plt.scatter(x, p50, marker='x')
    plt.title(f"{gpu}: Observed median vs PriceIQ-style bands")
    plt.ylabel("$/GPU-hour"); plt.xticks(rotation=45, ha='right'); plt.tight_layout(); plt.savefig(savepath); plt.close()

# -------------------------
# RUN
# -------------------------
if __name__ == "__main__":
    df0 = load_quotes(INPUT_FILES)
    df0 = prepare_for_model(df0)

    # time-based split if timestamp exists
    if df0['timestamp'].notna().any():
        df0 = df0.sort_values('timestamp'); cutoff = int(len(df0) * 0.8)
        train, test = df0.iloc[:cutoff].copy(), df0.iloc[cutoff:].copy()
    else:
        train, test = df0.copy(), None

    # tighten categories to what exists
    for c in ['price_type','region','cpu_platform','gpu_model']:
        train[c] = train[c].astype(str)

    formula = build_formula(INCLUDE_PROVIDER_FE, INCLUDE_DURATION_TERM)

    try:
        ols_model, X_train, y_train, design_info_ols = fit_ols(train, formula)
    except Exception:
        print("[OLS] Design failed; retrying simplified formula (no CPU terms/provider FE)")
        simple_rhs = "C(gpu_model) + C(price_type) + C(region) + np.log1p(duration_days)"
        y, X = dmatrices("price_per_gpu_hr ~ " + simple_rhs, data=train, return_type='dataframe')
        ols_model = sm.OLS(y, X).fit(cov_type='HC1')
        X_train, y_train = X, y
        design_info_ols = X.design_info
        formula = "price_per_gpu_hr ~ " + simple_rhs

    rhs = formula.split('~',1)[1]
    q_models, design_info_q = fit_quantiles(train, rhs, QUANTILES)

    evaluate_and_export(train, ols_model, X_train, y_train, q_models)

    grid = build_prediction_grid(train)
    preds = predict_with_models(grid, ols_model, q_models, design_info_ols=design_info_ols, design_info_q=design_info_q)
    preds.to_csv(OUT_DIR / 'predictions_grid.csv', index=False)

    if test is not None and not test.empty:
        X_test = build_design_matrices([design_info_ols], test, return_type='dataframe')[0]
        test['pred_ols'] = ols_model.predict(X_test)
        for tau, m in q_models.items():
            Xq = build_design_matrices([design_info_q], test, return_type='dataframe')[0]
            test[f'pred_q{int(tau*100)}'] = m.predict(Xq)
        test.to_csv(OUT_DIR / 'test_scored.csv', index=False)
        with open(OUT_DIR / 'metrics_test.json','w') as f:
            json.dump({'r2': float(r2_score(test['price_per_gpu_hr'], test['pred_ols'])),
                       'mae': float(mean_absolute_error(test['price_per_gpu_hr'], test['pred_ols']))}, f, indent=2)

    for gpu in ['H100','H200']:
        plot_price_bands(train, preds, gpu, OUT_DIR / f"bands_{gpu}.png")

    print(f"Artifacts written to: {OUT_DIR.resolve()}")


[load_quotes] Loaded sfcompute_grid/grid_history.csv -> 30 rows
[OLS] Design failed; retrying simplified formula (no CPU terms/provider FE)
OLS R^2=0.250  MAE=$0.01/GPU-hr  MAPE=0.94%
Artifacts written to: /Users/evieculloty/Documents/Forward Compute/docs/data/derived/priceiq_model
